In [20]:
#from sage.interfaces.gap import  get_gap_memory_pool_size, set_gap_memory_pool_size
#set_gap_memory_pool_size(24364842180)

'''Graded characters in the regular block O_0.
Note: Notation is dominant: L(e) is finite dimensional, Delta(e) is projective, Delta(w0) = L(w0), ...'''


W = WeylGroup("D6", prefix="s")
[s1,s2,s3,s4,s5,s6] = W.simple_reflections()

##################################################################################

n = rank(W)
w0 = W.long_element()
e = W(1)


####### Kazhdan-Lusztig polynomials ##########################

# A faster implementation of KL-polynomials (using the optional package Coxeter 3) is given by this
# Fokko Ducloux’s Coxeter3 C++ library.

# Had to install it: I just typed "sage -i coxeter3" in the terminal.

# It seems that one can direcly coerce from WeylGroup to CoxeterGroup and vice versa.
# I will therefore use CoxeterGroup to calculate KL-polynomials, but for all other Bruhat business I will use WeylGroup.

R.<q> = LaurentPolynomialRing(QQ)

KL = KazhdanLusztigPolynomial(W,q)  # KL-polynomials implemented in standard Sage way
# http://doc.sagemath.org/html/en/reference/combinat/sage/combinat/kazhdan_lusztig.html


CoxeterPackage = CoxeterGroup(W, implementation="coxeter3")

def KLP(x,y):
    '''Returns the KL-polynomial, implemented in "Coxeter3" package by Fokko du Cloux.
    http://math.univ-lyon1.fr/~ducloux/coxeter/coxeter3/english/coxeter3_e.html'''
    
    if x not in W:
        x = convert_from_123(x)
    if y not in W:
        y = convert_from_123(y)
    
    return CoxeterPackage.kazhdan_lusztig_polynomial(CoxeterPackage(x), CoxeterPackage(y))
    # If "coxeter3" is not installed, remove the line 'CoxeterPackage = CoxeterGroup(W, implementation="coxeter3")'
    # and in this function return KL.P(x,y)
    #return KL.P(x,y)

#Point:
#    - standard Sage way: KL.P(x,y)
#    - faster way: KLP(x,y) 


def mu(w,x):
    '''Returns the KL mu-function with arguments w,x.
    By Humphrey's BGG book p. 175 and p. 169, for w<x we have:
    mu(x,w) = mu(w0*w,w0*x),
    mu(w,x) = dim Exit^1 (L_w,L_x) = dim Exit^1(L_x,L_w) = dim Exit^1(Delta_x,L_w).'''

    if w not in W:
        w = convert_from_123(w)
    if x not in W:
        x = convert_from_123(x)
        
    if w.bruhat_le(x):
        poly_dict = KLP(w,x).dict()       
        j = (x.length()-w.length()-1)/2 
        if j not in poly_dict.keys():
            return 0
        return poly_dict[j]

    return 0


def dimExt(i,y,w):  
    '''Returns dimExt^i(Delta(y),L(w)).'''
    
    y = y*w0                               
    w = w*w0   
    poly_dict = KLP(y,w).dict()       
    j = (w.length()-y.length()-i)/2 
    if j not in poly_dict.keys():
        return 0
    return poly_dict[j]


def mult_Delta_L(y,w):
    '''Returns the Jordan-Holder multiplicity [Delta(y),L(w)] in O_0.
    This is quicker than the analogous operation on GradedChar classes because
    it does not calculate the full Delta.'''
      
    return KLP(y,w)(1)               


def mult_graded_Delta_L(w,k,x):
    '''Returns the graded Jordan-Holder multiplicity [Delta(w) shifted by k,L(x)].
    This is quicker than the analogous operation on GradedChar classes because
    it does not calculate the full Delta.''' 

    l_wx = x.length() - w.length()
    
    i = (l_wx-k)
    if i<0 or i%2!=0:
        return 0
    i = i/2
    
    KL_wx = KLP(w, x).dict()
    if i not in KL_wx.keys():
        return 0
    
    return KL_wx[i]


def earliest_occurence(x):
    '''Returns the smallest non-negative k such that L(x) occurs in Delta(e) at level k.
    To be used later in Lusztig's a function.'''
      
    for k in range(x.length()+1):
        
        if mult_graded_Delta_L(e,k,x) > 0:
            return k
    

##### Parabolic subgroups


def W_(Sigma):
    '''Parabolic subgroup W_Sigma as a list.'''
    
    return W.bruhat_interval(e,w0_(Sigma))


def minimal_rep(x,Sigma):
    '''The minimal representative of the class x*W_Sigma.'''
    
    return x.coset_representative(Sigma, side='right')


def w0_(Sigma):
    '''Returns the longest element of the parabolic subgroup defined by Sigma.
    Format of sigma: e.g. Sigma = [1,2,4] returns s4*s1*s2*s1.'''
    
    return (w0.coset_representative(Sigma, side='right')).inverse() * w0


def Maximal_elements_of_parabolics():
    '''Returns a list of the longest elements of all parabolic subgroups.'''
    
    return [w0_(Sigma) for Sigma in Subsets(range(1,rank(W)+1))]



######### Some operations on graded characters #########


def remove_keys_with_value(dict,value):
    '''Removes all items from dict with given value. Changes the original dict, does not return anything.
    Auxiliary function used in GradedChar().remove_zeros.'''
    
    remove = []
    for x in dict:
        if dict[x] == value:
            remove.append(x)
            
    for x in remove:
        dict.pop(x)

        
def remove_keys_with_value_smaller_than(dict,bound):
    '''Removes all items from dict with value smaller than or equal to "bound".
    Changes the original dict, does not return anything.
    Auxiliary function used in GradedChar().only_positive.'''
    
    remove = []
    for x in dict:
        if dict[x] <= bound:
            remove.append(x)
            
    for x in remove:
        dict.pop(x)

    
def format_star(string):
    '''Windows cannot have * in filenames, so need to use this function when saving files.
    This will replace "*" with "_".'''
    
    new_string = string.replace("*", "_")
    return new_string


def convert_to_123(w):
    '''Converts an element from W to the "123" string notation.
    Does not work with coefficients, as "convert_to_123_long".'''
    
    if w == W(1):
        return "e"
    
    return str(w).replace("s","").replace("*","")


def convert_from_123(string):
    '''Converts one element from W in the "123" string notation to the usual "s1*s2*s3" notation.'''
    
    if type(string)== Integer:
        string = str(string)
        
    if string == "e":
        return W(1)
    
    string = "*".join([char for char in string])
    
    for i in range(1,n+1):
        string = string.replace(str(i),"s%s"%i)
    
    return eval(string)


def convert_to_123_long(string):
    '''E.g. For input "4*s1*s2*s3", the output is "4*123".
    Not used for now.'''
    
    string_e = ""  # This should be a copy of string, but for "e" instead of each "1" that represent the trivial composition factor.
    
    for i in range(len(string)):
        
        condition = 0
        
        if string[i] == "1" and i>0:
            
            if string[i-1] not in ["s","-","0","1","2","3","4","5","6","7","8","9"]:
                
                if i+1==len(string):
                    condition = 1
                
                else:
                    if string[i+1] not in ["*",":","0","1","2","3","4","5","6","7","8","9"]:
                        condition = 1
    
        if condition == 0:
            string_e += string[i]
        
        else:
            string_e += "e"
    
        
    string_no_ast = re.sub(r'(?<=(?<=s).)\*',"",string_e) # Remove "*" if there is "s" two places before.
    string_no_ast_s = string_no_ast.replace("s","")
    
    return string_no_ast_s


class GradedChar:
    '''A class representing a graded character of module in the graded version of category O.
    self.graded is a dictionary where keys are indices of the graded components.
    Each graded component is a dictionary with keys being composition factors in that graded piece, and values are multiplicities.'''
    
    def __init__(self):
        self.component = {}
        self.name = ""
           
    def __str__(self):
        '''Each line becomes graded piece.'''
        
        if self == char_0():   # For zero character, print "0" instead of nothing.
            return "0"
        
        str_glob = ""
        if self.name != "":
            str_glob += self.name +":\n\n"
        
        for i in sorted(self.component.keys()):
            lis_i = []
            
            for w in (self.component[i]).keys():
                
                m = self.component[i][w]    # Multiplicity of w in i-th graded piece of self.
                m_string = ""
                if m != 1:
                    m_string = "%d*"%m      # Needed to print nicelly formated graded pieces.
                
                lis_i += ['%s'%m_string + str(w)]
            
            str_glob += "%d: "%i + ", ".join(lis_i) + "\n"
        
        return str_glob

    def __eq__(self, other): 
        if not isinstance(other, GradedChar):
            # Don't attempt to compare against unrelated types.
            return NotImplemented
        
        self.remove_zeros()      # Remove all the redundancies before comparing.
        other.remove_zeros()
        
        return self.component == other.component

    def __add__(self,char2):    
        char_sum = GradedChar()

        for k in (set(self.component.keys()) | set(char2.component.keys())):    # Union of keys without repeition.

            self_k = self.component.get(k,{})   # ".get" avoids KeyError
            char2_k = char2.component.get(k,{})

            char_sum.component[k] = {}

            for w in (set(self_k.keys()) | set(char2_k.keys())):

                char_sum.component[k][w] = self_k.get(w,0) + char2_k.get(w,0)

        return char_sum
    
    def __mul__(self,other):
        prod = GradedChar()

        for k in self.component:
            prod.component[k] = {}
        
            for w in self.component[k]:
                prod.component[k][w] = other * self.component[k][w]

        return prod
    
    def __rmul__(self,other):
        return self*other
    
    def __sub__(self,other):
        return self+((-1)*other)
    
    def __neg__(self):
        return (-1)*self
    
    
    def rename(self,new_name):
        '''Changes the name of self, and returns new_name.'''
        
        self.name = new_name
        return self.name
    
    
    def remove_zeros(self):
        '''Removes all composition factors with coefficients zero.
        These can occur only when subtracting, i.e. dealing with characters of virtual modules.'''
        
        for k in self.component:
            remove_keys_with_value(self.component[k],0)
        
        remove_keys_with_value(self.component,{})
        
        return self
        
    
    def only_positive(self):
        '''Removes all composition factors with non-positive coefficients.
        These can occur only when subtracting, i.e. dealing with characters of virtual modules.'''
        
        for k in self.component:
            remove_keys_with_value_smaller_than(self.component[k],0)
        
        remove_keys_with_value(self.component,{})
        
        return self    
    
    
    def add_factor(self,i,w):
        '''Adds a composition factor to self in i-th graded piece.'''
        
        if i not in self.component:
            self.component[i] = {}
        
        if w not in self.component[i]:
            self.component[i][w] = 0
        
        self.component[i][w] += 1
        
        self.name = ""    # The character "self" has changed, so the old name is not valid anymore.
        
        return self

    
    def is_simple(self):
        '''Checks whether self is a simple module.'''

        self.remove_zeros()

        if len(components(self)) != 1:
            return False

        only_component = self.component[ list(self.component.keys())[0] ]

        if len(only_component.keys()) != 1:
            return False

        only_factor = list(only_component.keys())[0]

        if only_component[ only_factor ]  != 1:
            return False

        return True
   
    
    def is_true_character(self):
        '''Returns True iff all the coefficients of self are non-negative.
        Assumes that all the coeffictients are integers.'''
        
        for i in self.component:
            for w in self.component[i]:
                if not ( self.component[i][w] >= 0 ):
                    return False
        return True
  

    def min_character(self,char2):
        '''Returns the character whose components have multiplicities equal to minimum
        of the multiplicities of the corresponding components in self and char2.'''
        
        char_min = GradedChar()
       
        for k in (set(self.component.keys()) | set(char2.component.keys())):    # Union of keys without repeition.

            self_k = self.component.get(k,{})   # ".get" avoids KeyError
            char2_k = char2.component.get(k,{})

            char_min.component[k] = {}

            for w in (set(self_k.keys()) | set(char2_k.keys())):

                char_min.component[k][w] = min(self_k.get(w,0),char2_k.get(w,0))

        return char_min    
    

    
######### Some operations on graded characters #########


def shift(char,i):
    '''Returns the shift by i of char.'''
    
    char_new = GradedChar()
    
    for key in char.component.keys():
        char_new.component[key-i] = char.component[key]
    
    return char_new


def dual(char):
    '''Returns the graded dual of char.
    Input: GradedChar.'''
    
    char_dual = GradedChar()
    
    for key in char.component.keys():
       char_dual.component[-key] = char.component[key]
    
    return char_dual


def components(char):
    '''Returns the list of indexes of non-zero graded pieces of char.'''
    
    char.remove_zeros()
    return sorted(char.component.keys())


def truncate(char,lis):
    '''Cuts off from char all graded pieces from lis.'''
    
    for k in lis:
        char.component.pop(k, None)
    
    return char
    
    
def mult(char,i,w):
    '''Returns the multiplicity of w in i-th graded piece of char.'''
    
    if i not in char.component:
        return 0
    
    return char.component[i].get(w,0)


def total_mult(char,w):
    '''Returns the total multiplicity of w in char.'''
    
    return sum(char.component[i].get(w,0) for i in char.component)


def dict_mult(char,w):
    '''Returns dictionary with items (k:m), where m is the multiplicity of w in k-th graded piece of char.'''
    
    dic = {}
    for k in char.component:
        if char.component[k].get(w,0) != 0:
            dic[k] = char.component[k][w]
            
    return dic


def clean(char):
    '''Returns cleaner output: simple reflections are denoted by 1,2,3, ...'''
    
    char_dummy = GradedChar()
    
    char_dummy.name = (char.name).replace("(1)","(e)").replace("s","").replace("*","")
    
    for k in char.component:
        char_dummy.component[k] = {}
        
        for w in char.component[k]:
            char_dummy.component[k][convert_to_123(w)] = char.component[k][w]
    
    return(str(char_dummy))


def print_clean(char):
    '''Prints cleaner output: simple reflections are denoted by 1,2,3, ...'''
    
    print(clean(char))
    

def combinatorial_cokernel(char1, char2):
    '''Returns the lower bound on the cokernel of a map from X to Y.'''
    
    virtual = char2-char1
    virtual.only_positive()
    
    return virtual


def combinatorial_kernel(char1, char2):
    '''Returns the lower bound on the kernel of a map from X to Y.'''
    
    virtual = char1-char2
    virtual.only_positive()
    
    return virtual



def ungrade(X):
    '''Returns the ungraded version of graded character X: all composition factors are moved to degree 0.'''
    
    unX = char_0()
    
    for i in components(X):
        unX += shift(X,i)
    
    truncate(unX,[i for i in components(unX) if i != 0])
    
    return unX



######### Some graded characters #########


def char_0():
    '''Returns the zero graded character.'''
    
    zero = GradedChar()
    zero.rename('0')
    return zero


def char_L(w):
    '''Returns the graded character of the simple L(w).'''
    
    L = GradedChar()
    L.add_factor(0,w)
    L.rename('L(%s)'%w)
    return L

        
def char_Delta(w):
    '''Returns the graded character of the Verma Delta(w).
    Reference: Humphrey's BGG book, Thm p. 175.'''
        
    Delta = GradedChar()
    
    const_name = 'Delta(%s)'%w
    const_dict = is_saved(const_name, option1="read_it")      # Here we check if it already exists in a file.
    
    if const_dict != None:
        Delta.component = const_dict
        Delta.rename(const_name)
        return Delta
    
    for x in W.bruhat_interval(w,w0):

        l_wx = x.length() - w.length()
        KL_wx = KLP(w, x).dict()
        for i in KL_wx.keys():
            for j in range(KL_wx[i]):
                Delta.add_factor(l_wx-2*i, x)

    Delta.rename(const_name)    
    return Delta
     

def char_Nabla(w):
    '''Returns the graded character of the dual Verma Nabla(w).'''
    
    Nabla = dual(char_Delta(w))
    Nabla.rename('Nabla(%s)'%w)
    return Nabla
    

def char_P(w):
    '''Returns the graded character of the indecomposable projective P(w).
    Uses graded BGG reciprocity.'''
    
    P = GradedChar()

    const_name = 'P(%s)'%w
    const_dict = is_saved(const_name, option1="read_it")      # Here we check if it already exists in a file.
    
    if const_dict != None:
        P.component = const_dict
        P.rename(const_name)
        return P
    
    for x in W.bruhat_interval(e,w):
        
        Delta = char_Delta(x)
        
        dic = dict_mult(Delta,w)
        
        for k in dic:
            P += shift(Delta,-k)*dic[k]

    P.rename('P(%s)'%w)
    return P
        
    
def char_I(w):
    '''Returns the graded character of the indecomposable injective I(w).'''
    
    I = dual(char_P(w))     
    I.rename('I(%s)'%w)
    
    return I
        

def char_T(w):
    '''Returns the graded character of the indecomposable tilting module T(w).
    Uses (graded) Soergel reciprocity.
    See Humphrey's BGG book, p. 233.
    This can also be proved by applying translation to graded BGG reciprocity.'''
    
    T = GradedChar()
    
    const_name = 'T(%s)'%w
    const_dict = is_saved(const_name, option1="read_it")      # Here we check if it already exists in a file.
    
    if const_dict != None:
        T.component = const_dict
        T.rename(const_name)
        return T
    
    for x in W.bruhat_interval(w,w0):
        dict = dict_mult(char_Delta(w0*x),w0*w)
        
        for k in dict:
            T += shift(char_Delta(x),k)*dict[k]
    
    T.rename('T(%s)'%w)
    return T    
    

def graded_char_BGG(w):
    '''Returns the graded character of the BGG complex of L(w).'''
    
    sum = char_0()
    l_w = w.length()
    
    for x in W.bruhat_interval(w,w0):
        l_x = x.length()
        
        sum += ((-1)^(l_x-l_w))*shift(char_Delta(x),-l_x+l_w)
        
    return sum



################ Projective functors ################


def theta_simple_simple(s,y):        
    '''Returns the graded character of theta_s(L_y), where s is a simple reflection,
    and y an element from W (not of class GradedChar).
    Reference: Coulembier-Mazorchuk-Zhang, Proposition 16,
    plus the fact that y and z are comparable if z appears.'''
        
    result = GradedChar()
    
    if y.bruhat_le(y*s):
        return result
    
    result.add_factor(-1,y)
    result.add_factor(1,y)
    result.add_factor(0,y*s)
    
    for z in W.bruhat_interval(y,w0):
        
        if z.bruhat_le(z*s):
            m = mu(y,z)
            if m != 0:
                result += m*char_L(z)
    
    return result


def theta_simple(s,M):
    '''Returns the graded character of theta_s(M), where s is a simple reflection,
    and M is from class GradedChar.
    Uses the function "theta_simple_simple" and the fact that theta is exact functor.'''
        
    result = GradedChar()
    
    for k in M.component:
        for w in M.component[k]:
            result += shift(M.component[k][w] * theta_simple_simple(s,w),  -k)
    
    return result


def theta(ws, char):          
    '''Returns the graded character of theta_ws(char).
    Uses recursion from [Mazorchuk: Some homolgical properties ... I, the proof of Theorem 11.a)].'''
    
    result = GradedChar()
    
    if char.name != "":
        const_name = 'theta_%s(%s)'%(ws,char.name)
        const_dict = is_saved(const_name, option1="read_it")      # Here we check if it already exists in a file.

        if const_dict != None:
            
            result.component = const_dict
            result.rename(const_name)

            return result
    
    factors = ws.reduced_word()
    
    if len(factors) == 0:    # theta_e = Identity
        return char
    
    if char == char_L(w0):   # theta_x(L(w0)) = T(w0*x)
        result = char_T(w0*ws)
        result.rename(const_name)
        return result
    
    s = W.simple_reflections()[factors[len(factors)-1]]
    
    if len(factors) == 1:    # basis of the recursion
        
        result = theta_simple(s,char)
        
        if char.name != "":
            result.rename(const_name)
        return result
    
    w = ws * s
        
    subtract = [ y for y in W.bruhat_interval(e,w) if (y*s).bruhat_le(y) and mu(y,w) !=0  ]
    
    result = theta_simple(s, theta(w,char))
    
    for y in subtract:
        result = result - mu(y,w)*theta(y,char)
    
    if char.name != "":
        result.rename(result.rename(const_name))
    return result


def M(x,y):
    '''Different notation for theta_x(L(y)).'''
    
    return theta(x,char_L(y))


def th(a,b):
    '''Same as theta, but uses 123-notation in number format, not as strings.
    Also prints theta along the way.'''
    
    x = convert_from_123(str(a))
    y = convert_from_123(str(b))
    
    X = theta(x,char_L(y))
    print_clean(X)

    return  X



######### Saving and reading characters to/from external files #########


import os
import datetime
import re    # Regular expressions!

    
def save(char):
    '''Saves char to a file in the folder.
    In fact, it saves 
    1. "char.component" dictionary to "./Graded_characters_in_O_0_data/Modules/'type'/", under the name "dict_'name'.txt",
    2. The output of print(char) string to "./Graded_characters_in_O_0_data/For_human/'type'/", under the name "'name'.txt".
    3. The output of print_clean(char) string to "./Graded_characters_in_O_0_data/For_human123/'type'/".
    Asterixes (*) are omitted from filenames.'''

    if char.name == "":
        return "Error: char has no name."        # For now I do not want to save characters without a name.
    
    CT = CartanType(W)[0]+str(CartanType(W)[1])
    
    folder1 = 'Graded_characters_in_O_0_data'
    For_computer = 'Modules'    
    path_computer = folder1 +'/' + For_computer     
    path_computer_CT = path_computer + '/' + CT
    
    for path in [folder1, path_computer, path_computer_CT]: # Check if necessary folders exist, and create them if not.

        if not os.path.isdir(path):  # Is there a folder already?
            os.mkdir(path)        # If not, create one.
    
    name_without_asterix = format_star(char.name)     # remove asterixes from name

    file_computer = open(path_computer_CT + "/dict_%s.txt"%name_without_asterix, "w+")        
    file_computer.write(str(char.component))     # Saves as a dictionary - usefull if I want to load back.    
    file_computer.close()  
    

def is_saved(name, option1):    # option1 should be "read_it" or "only_bool".
    '''Checks if the character with "name" has already been saved.
    If option1=="read_it": If no file, returns None. If yes, returns the dictionary.
    If option1=="only_bool": If no file, returns False. If yes, returns True.'''
    
    CT = CartanType(W)[0]+str(CartanType(W)[1])    # The names must be the same as in "save(char)" function.
    
    folder1 = 'Graded_characters_in_O_0_data'
    For_computer = 'Modules'
    
    name = format_star(name)    # remove asterixes from name
    
    path_computer_CT = folder1 +'/' + For_computer + '/' + CT + '/dict_' + name + '.txt'
    
    if not os.path.isfile(path_computer_CT):   # If there is no file.
        
        if option1 == "read_it":
            return None
        
        if option1 == "only_bool":
            return False
    
    # Here we assume that the file exists.
    
    if option1 == "only_bool":
        return True
    
    # Here option1 == "only_bool" and the file exists, so we want to read it and return it.
    
    file = open(path_computer_CT, "r")
    string = file.read()
    file.close()
    
    # https://stackoverflow.com/questions/11026959/writing-a-dict-to-txt-file-and-reading-it-back
    component = eval(string)
    
    # Unfortunately, the trivial element "W(1)"" is saved in .txt as "1".
    # So, after loading from .txt all its occurrences should be coerced back tp W(1)
    
    for k in component:
        for w in component[k]:
            if w == 1:
                component[k][W(1)] = component[k][Integer(1)]
                (component[k]).pop(Integer(1))
    
    return component


def save_all(kind):
    '''Saves to external files all standard object of the given kind = "Delta", "P", "T" or "thetaL".
    See function "save".
    In case thetaL, simple modules (i.e. theta_e(L(x))) are skipped.'''
    
    total = len(W)

    i=0
    print("Started: " + str(datetime.datetime.now()))  
    
    if kind != "thetaL":

        for w in W:
            if is_saved("%s(%s)"%(kind,w), option1="only_bool") == False:

                if kind == "Delta":
                    save(char_Delta(w))

                if kind == "P":
                    save(char_P(w))

                if kind == "T":
                    save(char_T(w))

            i+=1
            print(CartanType(W)[0]+str(CartanType(W)[1]) + " " + kind + ": " + str(i)+"/%d -"%total + str(datetime.datetime.now()))

    if kind == "thetaL":
        
        CT = CartanType(W)[0]+str(CartanType(W)[1])
        
        if not is_order_saved():
            print("Order not saved. Saving order first...")
            save_order()
            
        folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
        file =  CT + "_right_order.txt"
        path = folder + file
        if os.path.isfile(path):
            f = open(path, "r")
            R_order_string = f.read()
            f.close()
        print("Order loaded.")
    
        def le(x):
            return x.length()

        W_copy = list(W)
        W_copy.sort(key=le)    
        print("W sorted.")
    
        i = 0

        for x in W_copy:
            if x != e:
                for y in W_copy:
                    if "(%s, %s)"%(convert_to_123(x),convert_to_123(y.inverse())) in R_order_string:
                        if not is_saved( "theta_%s(L(%s))"%(x,y) , "only_bool"):
                            save(theta(x,char_L(y)))
                            print(CT + ": saved theta_%s(L(%s))"%(x,y))
                            print("%d/%d - "%(i,total) + str(datetime.datetime.now()))
            i += 1
            print("*"*20 + "\n%d/%d: x=%s done!\n"%(i,total,convert_to_123(x)) + "*"*20)
          
    print ("Finished with %s's!"%kind)
    return
    
    
    
######################## KL cells ###############################


def DR(w):
    '''Returns the set of simple right descents of w.'''
    
    return {W.simple_reflections()[i] for i in w.descents()}


def DL(w):
    '''Returns the set of simple left   descents of w.'''
        
    return DR(w.inverse())


def AL(w):
    '''Returns the set of simple left ascends of w.'''
    
    DescLe = list(DL(w))
    AscLe = [s for s in W.simple_reflections() if s not in DescLe]
    return set(AscLe)


def AR(w):
    '''Returns the set of simple right ascends of w.'''
    
    DescRi = list(DR(w))
    AscRi = [s for s in W.simple_reflections() if s not in DescRi]
    return set(AscRi)



######### In type A only:


def check_if_type_A():
    '''Used to block usage of RS correspondence outside od type A.'''
    
    if CartanType(W)[0] != 'A':
        raise ValueError("This function uses the RS correspondence, hence works only in type A!")
    

def to_perm(w):
    '''Converts an element from W to a permutation of type list.
    Mind that it is inverted in the proces.'''
    
    check_if_type_A()
    
    if w==1: w=W(1)
        
    return list((w.inverse()).to_permutation())


def to_W(t):
    '''The inverse of "to_perm".'''
    
    check_if_type_A()
    
    t = Permutation(list(t))
    red = prod(W.simple_reflections()[i] for i in t.reduced_word())
    
    if red==1: red=W(1)
    red = red.inverse()
    return red


######### In any type, but faster in A:

def KL_graph_onesided(side):
    '''Here side = "left" or "right".
    Returns a directed graph, which is almost exacly the left colored Kazhdan-Lusztig graph,
    reference: Björner-Brenti p. 176,
    but without loops and without labels.
    Used later to give preorders, and cells in types other than A.'''
    
    if side in ["l", "left", "L", "Left"]:
        side = 0
        
    if side in ["r", "right", "R", "Right"]:
        side = 1
        
    L = {}

    for x in W:
        L[x] = []
        lx = x.length()

        if side == 0:       # Björner-Brenti Lemma 6.2.4.
            for s in DL(x):
                L[x].append(s*x)

        if side == 1:
            for s in DR(x):
                L[x].append(x*s)

        for y in W.bruhat_interval(x,w0):
            ly = y.length()

            if (ly-lx)%2==1:   # Björner-Brenti Lemma 6.2.2.(i)


                if (side == 0 and DL(x).difference(DL(y)) != set()) or (side == 1 and DR(x).difference(DR(y)) != set()):
                    if mu(x,y) != 0:

                        L[x].append(y)

    return DiGraph(L)
    

def KL_graph(side):
    '''Here side = "left" or "right" or "twosided".
    Returns a directed graph, which is almost exacly the left colored Kazhdan-Lusztig graph,
    reference: Björner-Brenti p. 176,
    but without loops and without labels.
    Used later to give preorders, and cells in types other than A.'''
    
    if side in ["l", "left", "L", "Left"]:
        return KL_graph_onesided("left")
        
    if side in ["r", "right", "R", "Right"]:
        return KL_graph_onesided("right")
    
    L = KL_graph_onesided("left")    
    R = KL_graph_onesided("right")
    
    L.add_edges(R.edges())          # In two-sided case, we add all edges from left and right KL graph.
    
    return L     
    

def cell(side, w):
    '''Returns the left- or right- or 2-sided- KL cell of w.
    Use cell("left", w) for the left, cell("right", w) for the right cell,
    and cell("twoside", w) for the two-sided cell.'''
    
    if CartanType(W)[0] == 'A':   # In type A we use the RS-correspondence.
    
        RS_index = 2 # Unless said otherwise, we work with 2-sided cells.

        if side in ["left", "l", "Left", "L"]:
            RS_index = 0
        if side in ["right", "r", "Right", "R"]:
            RS_index = 1

        w = to_perm(w)  # Convert to permutation, to apply.

        lis = []

        if RS_index in [0,1]: # If we are dealing with 1-sided cells.

            for x in Permutations(range(1,n+2)):
                if RSK(x)[RS_index] == RSK(w)[RS_index]:
                    lis.append(Permutation(x))

            lis_W = [to_W(x) for x in lis]  # Convert back to W.

            return lis_W

        # Now we are dealing with 2-sided cells.

        for x in Permutations(range(1,n+2)):
            if RSK(x)[0].shape() == RSK(w)[0].shape():
                lis.append(Permutation(x))

        lis_W = [to_W(x) for x in lis]  # Convert back to W.

        return lis_W
    
    return KL_graph(side).strongly_connected_component_containing_vertex(w)   # In types other than A we use way slower procedure.
    
    
def are_cells_saved():
    '''Return "True" iff left cells are saved.'''
    
    folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
    file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_left_cells.txt"
    path = folder + file
    return os.path.isfile(path)


def L_cell(w):
    '''Returns the left-KL cell of w.'''
    
    if w==e:
        return [e]
    
    if are_cells_saved():
        folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
        file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_left_cells.txt"
        path = folder + file
        f = open(path, "r")
        contents = f.read()
        f.close()
        LC_123 = eval(contents)
                
        w_123 = eval(convert_to_123(w))
             
        for cel in LC_123:
            if w_123 in cel:
                return [convert_from_123(str(x)) for x in cel]
            
    return cell("left", w)
    

def R_cell(w):
    '''Returns the right-KL cell of w.'''
    
    return [x.inverse() for x in L_cell(w.inverse())]


def two_cell(w):
    '''Returns the 2-sided-KL cell of w.'''
    
    Lw = L_cell(w)
    result = []
    
    for x in Lw:
        Rx = R_cell(x)
        for y in Rx:
            if y not in result:
                result.append(y)
    return result    


def cells(side):
    '''Returns the list of all left, right or two-sided cells, depending on the argument side.'''
    
    if CartanType(W)[0] == 'A':   # In type A we use the RS-correspondence.
    
        remainder = set(W)
        partition = []

        while len(remainder) != 0:

            w = remainder.pop()

            cellw = cell(side, w)

            remainder = remainder.difference(set(cellw))
            partition.append(cellw)

        return partition
    
    return KL_graph(side).strongly_connected_components()   # In types other than A we use way slower procedure.
    

def save_cells():
    '''Calculates and saves left cells.'''
    
    print("Cells already saved: %s"%are_cells_saved())
    CL = cells("left")

    def eval_number(st): # Need this to remove quotation marks from 123-notation, except for "e".
        if st == "e":
            return st
        return eval(st)

    result = [set( eval_number(convert_to_123(x)) for x in C)  for C in CL]
    
    CT = CartanType(W)[0]+str(CartanType(W)[1])
    f = open("Graded_characters_in_O_0_data/Cells_and_orders/%s_left_cells.txt"%CT,"w")
    f.write(str(result))
    f.close()
    print("Cells now saved: %s"%are_cells_saved())
    return


def cells_graph(side):
    '''Returns directed graph where nodes are side-cells, and arrows are side-order,
    where side = "left", "right" or "twoside".'''
    
    return KL_graph(side).strongly_connected_components_digraph()  
    

def two_smaller(w,v):
    '''Checks if w is smaller than or equall to v in the two sided order.
    Uses RS-correspondence, therefore works only in type A.'''
    
    check_if_type_A()
    
    w_partition = RSK(to_perm(w))[0].shape()
    v_partition = RSK(to_perm(v))[0].shape()
    
    return w_partition.dominates(v_partition)



def is_order_saved():
    '''Return "True" iff right order is saved.'''
    
    folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
    file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_right_order.txt"
    path = folder + file
    return os.path.isfile(path)


def L_smaller(x,y):
    '''Checks whether x is left-smaller than or left-equivalent to y, for x,y in W.
    For this the line "#all_paths = cells_graph("left").all_simple_paths(trivial=True)" must be uncommented,
    but this may take forever to load.
    In general, extremly inefficient.'''
    
    if x == e or y == w0:
        return True
    
    if (x == w0 and y != w0) or (y == e and x != e):
        return False
    
    if is_order_saved(): # If it is already saved:
        folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
        file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_right_order.txt"
        path = folder + file

        f = open(path, "r")
        R_order_string = f.read()
        f.close()  
        if "(%s, %s)"%(convert_to_123(x.inverse()),convert_to_123(y.inverse())) in R_order_string:
            return True
        else:
            return False    
       
    Lcell_x = set(cell("left",x))
    Lcell_y = set(cell("left",y))
    
    if Lcell_x == Lcell_y:
        return True

    x_less_y = False

    for path in all_paths:
        
        if path[0] == Lcell_y and path[-1] == Lcell_x:
            x_less_y = True
            break
            
    return x_less_y


def strictly_L_smaller(x,y):
    '''Checks whether x is strictly left-smaller than y, for x,y in W.
    For this the line "#all_paths = cells_graph("left").all_simple_paths(trivial=True)" must be uncommented,
    but this may take forever to load.
    In general, extremly inefficient.'''

    if x == e:
        if y == e:
            return False
        else:
            return True
    if y == w0:
        if x == w0:
            return False
        else:
            return True        
        
    if is_order_saved(): # If it is already saved:
        
        folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
        file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_right_order.txt"
        path = folder + file

        if L_smaller(x,y) and not L_smaller(y,x):
            return True
        else:
            return False    
    
    Lcell_x = set(cell("left",x))
    Lcell_y = set(cell("left",y))

    if Lcell_x == Lcell_y:
        return False
    
    x_less_y = False

    for path in all_paths:
        
        if path[0] == Lcell_y and path[-1] == Lcell_x:
            x_less_y = True
            break
            
    return x_less_y


def R_smaller(x,y):
    '''Checks whether x is right-smaller than or right-equivalent to y, for x,y in W.
    For this the line "#all_paths = cells_graph("left").all_simple_paths(trivial=True)" must be uncommented,
    but this may take forever to load.
    In general, extremly inefficient.'''
       
    return L_smaller(x.inverse(),y.inverse())


def strictly_R_smaller(x,y):
    '''Checks whether x is strictly left-smaller than y, for x,y in W.
    For this the line "all_paths = cells_graph("left").all_simple_paths(trivial=True)" must be uncommented,
    but this may take forever to load.
    In general, extremly inefficient.'''
    
    return strictly_L_smaller(x.inverse(),y.inverse())


def save_order():
    '''Calculates and saves right order.'''
    
    print("Order already saved: %s"%is_order_saved())
    
    # This could take forever to calculate. There probably exists more efficient way.
    all_paths = cells_graph("left").all_simple_paths(trivial=True) 

    def eval_number(st): # Need this to remove quotation marks from 123-notation, except for "e".
        if st == "e":
            return st
        return eval(st)

    result = set()

    for path in all_paths:
        Y = path[0]
        X = path[-1]
        for x in X:
            for y in Y:
                result.add( (eval_number(convert_to_123(x.inverse())),eval_number(convert_to_123(y.inverse()))) )
    
    CT = CartanType(W)[0]+str(CartanType(W)[1])
    f = open("Graded_characters_in_O_0_data/Cells_and_orders/%s_right_order.txt"%CT,"w")
    f.write(str(result))
    f.close()
    print("Order now saved: %s"%is_order_saved())
    return

    
######################## Lusztig's a function ###############################


def is_involution(w):
    '''Checks if w is an involution.'''
    
    return w == w.inverse()


def Duflo_involution(w, side):
    '''Returns the unique (Duflo) involution from the side="left" or "right" cell of w.'''
        
    if CartanType(W)[0] == 'A':
    
        cellw = cell(side,w)

        for x in cellw:

            if is_involution(x):
                break

        return x
    
    # Much slower for other types, loads "L_cell()" and "Duflo_Involutions()".
    if side == "right":
        return list(set(R_cell(w)).intersection(Duflo_Involutions()))[0]
    
    if side == "left":
        return list(set(L_cell(w)).intersection(Duflo_Involutions()))[0]

    
def Duflo_Involutions():
    '''Returns the list of Duflo involutions.'''
     
    # If it is already saved
    if are_Duflo_involutions_saved():
        folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
        file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_Duflo_involutions.txt"
        path = folder + file
        f = open(path, "r")
        contents = f.read()
        f.close()
        D = [convert_from_123(str(x)) for x in eval(contents)]
        return D    

    Involutions = {x for x in W if is_involution(x)}
    
    if CartanType(W)[0] == 'A':
        return list(Involutions)  
    
    lis = []

    for cell in cells("l"):

        a = w0.length()
        cand = w0

        for x in (set(cell)).intersection(Involutions):

            ax = earliest_occurence(x)

            if ax < a:
                a = ax
                cand = x

        lis.append(cand)
    
    return lis


def are_Duflo_involutions_saved():
    '''Return "True" iff Duflo involutions are saved.'''
    
    folder = 'Graded_characters_in_O_0_data/Cells_and_orders/'
    file =  CartanType(W)[0]+str(CartanType(W)[1]) + "_Duflo_involutions.txt"
    path = folder + file
    return os.path.isfile(path)


def save_Duflo_involutions():
    '''Calculates and saves Duflo involutions.'''
    
    print("Duflo involutions already saved: %s"%are_Duflo_involutions_saved())
    D = Duflo_Involutions()
    
    def l(x):   # Needed to sort D
        return x.length()
    D.sort(key = l)
    
    def eval_number(st): # Need this to remove quotation marks from 123-notation, except for "e".
        if st == "e":
            return st
        return eval(st)
    
    result = [eval_number(convert_to_123(x)) for x in D]
    
    CT = CartanType(W)[0]+str(CartanType(W)[1])
    f = open("Graded_characters_in_O_0_data/Cells_and_orders/%s_Duflo_involutions.txt"%CT,"w")
    f.write(str(result))
    f.close()
    print("Duflo involutions now saved: %s"%are_Duflo_involutions_saved())
    return


def a(w):
    '''Returns the value of Lusztig's a function on w.
    Much faster in type A.'''
    
    if CartanType(W)[0] == 'A':
    
        if is_involution(w):
            return earliest_occurence(w)
    
    d = Duflo_involution(w,"right")
    
    return earliest_occurence(d)
    


def b(x,y):
    '''Returns the function b(x,y) from "SHPOV".'''
    
    if R_smaller(x,y.inverse()):
        return max(components( theta(x,char_L(y)) ))
    else:
        return -1 # Or some very negative number.


################## Delta flags ##################

def Delta_flag(X):
    '''Returns a dicionary where keys are graded degrees, and values are dictionaries
    where keys are parameters of Deltas and values are multiplicities in the given degree.'''
    
    if X == char_0():
        return {}
    
    if not X.is_true_character():
        raise TypeError("Does not have a Delta flag!") 
    

    result = {}  
    X_ = deepcopy(X)
    top_degree = min(components(X))
    top = deepcopy(X)
    truncate(top, [i for i in components(X) if i != top_degree] )   
    result.update(top.component)
    
    
    for w in top.component[top_degree]:
        X_ -=   shift(top.component[top_degree][w] * char_Delta(w), -top_degree)
    X_.remove_zeros()
    
    result.update(Delta_flag(X_))
    
    return result


def print_Delta_flag(X):
    '''Prints out the parameters of Delta flag of a graded character X.'''
    
    result = char_0()
    result.component = Delta_flag(X)
    result.name = "Delta flag of %s"%X.name
    print_clean(result)




print(DynkinDiagram(W))
    

    


            O 6
            |
            |
O---O---O---O---O
1   2   3   4   5   
D6


In [10]:
############### JOINS ###############              
                
# W_poset = W.bruhat_poset()

def W_subposet(L):
    '''This is equivalent to W.bruhat_poset().subposet(),
    but without calculating the full W.bruhat_poset().'''
    
    def br_le(x,y):
        return x.bruhat_le(y)    
    return Poset((L,br_le))




def join(S):
    SS = [convert_from_123(a) for a in S if a not in W] + [a for a in S if a in W]
    
    U = set(W.bruhat_interval(SS[0],w0))
    for a in SS[1:]:
        U = U.intersection(set(W.bruhat_interval(a,w0)))
        
    minU = (W_subposet(list(U))).minimal_elements()
    
    if len(minU)==1:
        j = minU[0]
        return eval(convert_to_123(str(j)))
    else:
        return [eval(convert_to_123(x)) for x in minU]

    
def join_B(S):
    '''Here should S consists of join-irreds with fixed descents.'''
    
    SS = [convert_from_123(a) for a in S if a not in W] + [a for a in S if a in W]
    bigrassmannians_fixed = [x for x in bigrassmannians if DL(x)==DL(SS[0]) and DR(x)==DR(SS[0])]
    
    U = set(x for x in bigrassmannians_fixed if (SS[0]).bruhat_le(x) )
    for a in SS[1:]:
        U = U.intersection(set(x for x in bigrassmannians_fixed if a.bruhat_le(x) ))
        
    minU = (W_subposet(list(U))).minimal_elements()
    
    if len(minU)==1:
        j = minU[0]
        return eval(convert_to_123(str(j)))
    else:
        return 0
        #return [eval(convert_to_123(x)) for x in minU]
    
    
def JM(w):
    if w not in W:
        w = convert_from_123(w)
    result = W_subposet([x for x in join_irreducibles if x.bruhat_le(w) ]).maximal_elements()
    return [convert_from_123(convert_to_123(x)) for x in result]

def BM(w):
    if w not in W:
        w = convert_from_123(w)
    return W_subposet([x for x in bigrassmannians if x.bruhat_le(w) ]).maximal_elements()

def JM2(w):
    if w not in W:
        w = convert_from_123(w)
    DLw = DL(w)
    DRw = DR(w)
    result = []
    for x in JM(w):
        x = convert_from_123(convert_to_123(x))
        if list(DL(x))[0] in DLw and list(DR(x))[0] in DRw:
            result.append(x)
    return result

def JM3(w):
    if w not in W:
        w = convert_from_123(w)
    DLw = DL(w)
    DRw = DR(w)
    result = W_subposet([x for x in join_irreducibles if x.bruhat_le(w) and (list(DL(x))[0] in DLw) and (list(DR(x))[0] in DRw) ]).maximal_elements()   
    return [convert_from_123(convert_to_123(x)) for x in result]    
    




In [38]:
######################## B2 ########################

#sum_J KL = 6

#bigrassmannians = join_irreducibles = dissectors: 6

bigrassmannians = [s2, s1, s1*s2*s1, s1*s2, s2*s1, s2*s1*s2]

join_irreducibles = [x for x in bigrassmannians]

dissectors = [x for x in join_irreducibles]

In [45]:
######################## G2 ########################

#sum_J KL = 10

#bigrassmannians = join_irreducibles = dissectors: 10

bigrassmannians = [s2*s1*s2*s1, s1*s2*s1*s2, s2, s2*s1*s2*s1*s2, s1*s2*s1, s1, s1*s2*s1*s2*s1, s2*s1*s2, s1*s2, s2*s1]

join_irreducibles = [x for x in bigrassmannians]

dissectors = [x for x in join_irreducibles]

In [52]:
######################## B3 ########################

#sum_J KL = 19

#bigrassmannians = join_irreducibles = dissectors: 19

bigrassmannians = [s3, s3*s2, s3*s2*s1, s3*s2*s3, s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3, s2, s2*s3, s2*s3*s2, s2*s3*s2*s1, s2*s1, s2*s3*s1*s2, s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s2, s1*s2*s3*s2*s1]

join_irreducibles = [x for x in bigrassmannians]

dissectors = [x for x in join_irreducibles]


In [59]:
######################## B4 ########################

#sum_J KL = 44

#bigrassmannians: 45

bigrassmannians = [s2*s3*s1*s2, s4*s3*s4*s2*s3*s4*s1*s2*s3*s4, s2*s1, s2*s3, s1*s2, s3*s2, s3*s4*s2*s3*s4*s1*s2*s3, s1, s3, s1*s2*s3, s3*s2*s1, s3*s4*s2*s3*s4*s2*s3*s1*s2, s3*s4*s2*s3*s4, s2, s3*s4*s2*s3*s4*s2*s3, s2*s3*s4*s1*s2*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s4, s4*s3*s4, s4*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s4*s2*s3, s4*s3*s4*s2*s3, s1*s2*s3*s4*s3*s2*s1, s4*s3*s4*s2*s3*s4*s1*s2*s3, s3*s4*s3, s3*s4*s3*s2*s1, s1*s2*s3*s4*s3, s2*s3*s4*s3*s2, s4, s3*s4*s2*s3*s4*s1*s2*s3*s4, s2*s3*s4, s4*s3*s2, s1*s2*s3*s4*s3*s2, s4*s3, s4*s3*s4*s2*s3*s4, s1*s2*s3*s4, s3*s4*s2*s3*s4*s1*s2*s3*s4*s1*s2*s3, s4*s3*s2*s1, s2*s3*s4*s3*s2*s1, s2*s3*s4*s1*s2*s3, s3*s4, s3*s4*s2*s3*s1*s2, s2*s3*s4*s3*s1*s2, s3*s4*s2*s3, s2*s3*s4*s3, s3*s4*s3*s2]

#join_irreducibles = dissectors: 44

join_irreducibles = [s4, s3, s2, s1, s3*s2*s1, s1*s2*s3, s2*s3*s4*s3*s2, s3*s4*s2*s3*s4*s1*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s3*s2*s1, s3*s4*s3*s2*s1, s1*s2*s3*s4*s3, s4*s3*s4*s2*s3*s4, s2*s3*s1*s2, s3*s4*s2*s3, s4*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s4, s3*s4*s3, s4*s3*s4*s2*s3, s2*s1, s3*s4*s2*s3*s4, s1*s2, s2*s3*s4*s3, s4*s3*s2*s1, s3*s4*s3*s2, s1*s2*s3*s4, s2*s3*s4*s3*s2*s1, s3*s4*s2*s3*s4*s2*s3*s1*s2, s3*s2, s2*s3*s4*s1*s2*s3*s4*s2*s3, s2*s3, s1*s2*s3*s4*s3*s2, s2*s3*s4*s1*s2*s3, s4*s3*s2, s3*s4*s2*s3*s4*s1*s2*s3, s2*s3*s4, s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s4*s2*s3*s1*s2, s4*s3*s4, s3*s4*s2*s3*s4*s2*s3, s4*s3, s3*s4*s2*s3*s4*s1*s2*s3*s4, s4*s3*s4*s2*s3*s4*s1*s2*s3*s4, s3*s4, s4*s3*s4*s2*s3*s4*s1*s2*s3]

dissectors = [x for x in join_irreducibles]

In [3]:
######################## D4 ########################

#sum_J KL = 28

#bigrassmannians: 30

bigrassmannians = [s4*s2*s3*s1*s2*s4, s2*s4*s1*s2, s2*s3*s1*s2, s2*s4*s3*s2, s2*s1, s2*s4, s2*s3, s1*s2, s4*s2, s1*s2*s4*s3*s2*s1, s3*s2*s4*s1*s2*s3, s3*s2, s1, s4, s2*s4*s1*s2*s3, s1*s2*s4*s3*s2, s2*s3*s1*s2*s4*s2*s3*s1*s2, s4*s2*s1, s1*s2*s4, s1*s2*s3, s2, s3, s3*s2*s4*s1*s2, s4*s2*s3*s1*s2, s2*s4*s3*s2*s1, s2*s3*s1*s2*s4, s3*s2*s1, s3*s2*s4, s4*s2*s3, s2*s4*s3*s1*s2]

#join_irreducibles: 29

join_irreducibles = [s3, s4, s2, s1, s2*s4*s3*s2, s3*s2*s1, s2*s4*s1*s2, s1*s2*s3, s4*s2*s1, s1*s2*s4, s3*s2*s4, s1*s2, s2*s4, s3*s2*s4*s1*s2*s3, s1*s2*s4*s3*s2*s1, s4*s2*s3, s3*s2, s4*s2, s4*s2*s3*s1*s2*s4, s1*s2*s4*s3*s2, s2*s1, s4*s2*s3*s1*s2, s3*s2*s4*s1*s2, s2*s4*s1*s2*s3, s2*s3*s1*s2, s2*s3*s1*s2*s4, s2*s4*s3*s2*s1, s2*s3, s2*s3*s1*s2*s4*s2*s3*s1*s2]

#dissectors: 26

dissectors = [s4*s2*s3*s1*s2*s4, s2*s1, s2*s4, s2*s3, s1*s2, s4*s2, s1*s2*s4*s3*s2*s1, s3*s2*s4*s1*s2*s3, s3*s2, s1, s4, s2*s4*s1*s2*s3, s1*s2*s4*s3*s2, s2*s3*s1*s2*s4*s2*s3*s1*s2, s4*s2*s1, s1*s2*s4, s1*s2*s3, s2, s3, s3*s2*s4*s1*s2, s4*s2*s3*s1*s2, s2*s4*s3*s2*s1, s2*s3*s1*s2*s4, s3*s2*s1, s3*s2*s4, s4*s2*s3]


In [3]:
######################## F4 ########################

#sum_J KL = 92

#bigrassmannians: 108

bigrassmannians = [s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s3*s2*s3, s4*s3*s2*s3*s4, s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4, s4*s3*s2*s3*s4*s1*s2*s3, s4*s3*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s4*s1*s2*s3*s2*s1, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3, s3*s4, s3*s2, s3*s4*s2*s3, s3*s4*s2*s3*s2, s3*s4*s2*s3*s2*s1, s3*s2*s1, s3*s4*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4, s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3, s3*s2*s3*s4, s3*s2*s3*s4*s3*s2*s3, s3*s2*s3*s4*s3*s2*s3*s1*s2, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s4*s3*s1*s2*s3, s3*s2*s3*s4*s3*s1*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3*s4*s3*s1*s2*s3, s3*s2*s3*s1*s2, s3*s2*s3*s4*s1*s2*s3, s3*s2*s3*s4*s1*s2*s3*s2, s3*s2*s3*s4*s1*s2*s3*s2*s1, s3*s4*s2*s3*s1*s2*s3*s4*s3*s1*s2*s3*s1*s2, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s4*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2, s2*s3, s2*s3*s4, s2*s3*s2, s2*s3*s4*s2*s3, s2*s3*s4*s2*s3*s2, s2*s3*s4*s2*s3*s2*s1, s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3*s4, s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s4*s1*s2*s3*s2, s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s4*s1*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s1*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s1*s2*s3, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s2, s1*s2*s3*s4*s2*s3, s1*s2*s3*s4*s2*s3*s2, s1*s2*s3*s4*s2*s3*s2*s1, s1*s2*s3*s2*s1, s1*s2*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4, s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1]

#join_irreducibles: 96

join_irreducibles = [s3, s2, s1, s4, s3*s4, s2*s3, s1*s2, s2*s3*s2, s3*s4*s2*s3, s2*s3*s2*s1, s1*s2*s3, s1*s2*s3*s2, s1*s2*s3*s4, s3*s4*s2*s3*s2, s3*s2*s3*s4*s3*s2*s3, s3*s4*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3, s2*s3*s1*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s2, s3*s4*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3*s2*s1, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s4*s3, s2*s3*s4, s2*s3*s4*s2*s3, s2*s3*s4*s2*s3*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4, s4*s3*s2, s4*s3*s2*s3, s1*s2*s3*s2*s1, s2*s3*s4*s2*s3*s2*s1, s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s4*s3*s2*s1, s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s4*s3*s2*s3*s4, s4*s3*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s4*s3*s2*s3*s4*s1*s2*s3*s2*s1, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2, s3*s2*s3, s3*s4*s2*s3*s2*s1, s3*s2*s3*s4*s3*s2*s3*s1*s2, s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s4, s3*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s3*s2*s3*s4*s1*s2*s3*s2, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s3*s2*s1, s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2*s1, s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4, s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2*s3*s4*s1*s2*s3*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2]

#dissectors: 80

dissectors = [s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s3*s2*s3, s4*s3*s2*s3*s4, s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4, s4*s3*s2*s3*s4*s1*s2*s3, s4*s3*s2*s3*s4*s1*s2*s3*s2, s4*s3*s2*s3*s4*s1*s2*s3*s2*s1, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3, s3*s4, s3*s2, s3*s4*s2*s3, s3*s4*s2*s3*s2, s3*s4*s2*s3*s2*s1, s3*s2*s1, s3*s4*s2*s3*s1*s2*s3*s4, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3, s3*s2*s3*s4, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s1*s2, s3*s2*s3*s4*s1*s2*s3*s2*s1, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s3*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2, s2*s3, s2*s3*s4, s2*s3*s2, s2*s3*s4*s2*s3, s2*s3*s4*s2*s3*s2, s2*s3*s4*s2*s3*s2*s1, s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2*s3*s4, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4, s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1, s2*s3*s4*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3, s2*s3*s1*s2*s3*s4*s3*s2*s3*s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s2, s1*s2*s3*s4*s2*s3, s1*s2*s3*s4*s2*s3*s2, s1*s2*s3*s4*s2*s3*s2*s1, s1*s2*s3*s2*s1, s1*s2*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4, s1*s2*s3*s4*s2*s3*s1*s2*s3*s1*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2, s1*s2*s3*s4*s2*s3*s1*s2*s3*s4*s1*s2*s3*s2*s1]


In [66]:
######################## B5 ########################

#sum_J KL = 85

#bigrassmannians: 90

bigrassmannians = [s3, s3*s2, s3*s4, s3*s2*s1, s3*s4*s5, s3*s4*s2*s3, s3*s4*s5*s4, s3*s4*s5*s4*s3, s3*s4*s2*s3*s1*s2, s2, s2*s3, s2*s3*s4, s2*s3*s4*s5, s2*s3*s4*s5*s4, s2*s3*s4*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s5, s4*s5*s3*s4, s4*s5*s3*s4*s5, s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s4, s4*s5*s4*s3, s4*s5*s4*s3*s2, s4*s5*s4*s3*s2*s1, s4*s5*s3*s4*s5*s3*s4, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s4*s5, s1*s2*s3*s4*s5*s4, s1*s2*s3*s4*s5*s4*s3, s5, s5*s4, s5*s4*s3, s5*s4*s3*s2, s5*s4*s3*s2*s1, s5*s4*s5, s5*s4*s5*s3*s4, s5*s4*s5*s3*s4*s5, s5*s4*s5*s3*s4*s2*s3, s5*s4*s5*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s5*s4*s3*s2, s1*s2*s3*s4*s5*s4*s3*s2*s1, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5, s5*s4*s5*s3*s4*s5*s2*s3*s4, s5*s4*s5*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4, s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s5*s4*s1*s2*s3, s2*s3*s4*s5*s4*s3*s1*s2, s4*s5*s3*s4*s5*s2*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s3*s4*s5*s2*s3*s4*s5, s4*s5*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s4*s3*s2, s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3, s3*s4*s5*s2*s3*s4*s5*s3*s4, s3*s4*s5*s2*s3*s4*s5, s3*s4*s5*s2*s3*s4, s3*s4*s5*s4*s2*s3, s3*s4*s5*s4*s3*s2, s3*s4*s5*s4*s3*s2*s1, s3*s4*s5*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s5*s2*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s4*s3*s2*s1, s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4*s5*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s2*s3*s4, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5]

#join_irreducibles = dissectors: 85

join_irreducibles = [s3, s3*s2, s3*s4, s3*s2*s1, s3*s4*s5, s3*s4*s2*s3, s3*s4*s5*s4, s3*s4*s5*s4*s3, s3*s4*s2*s3*s1*s2, s2, s2*s3, s2*s3*s4, s2*s3*s4*s5, s2*s3*s4*s5*s4, s2*s3*s4*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s5, s4*s5*s3*s4, s4*s5*s3*s4*s5, s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s4, s4*s5*s4*s3, s4*s5*s4*s3*s2, s4*s5*s4*s3*s2*s1, s4*s5*s3*s4*s5*s3*s4, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s4*s5, s1*s2*s3*s4*s5*s4, s1*s2*s3*s4*s5*s4*s3, s5, s5*s4, s5*s4*s3, s5*s4*s3*s2, s5*s4*s3*s2*s1, s5*s4*s5, s5*s4*s5*s3*s4, s5*s4*s5*s3*s4*s5, s5*s4*s5*s3*s4*s2*s3, s5*s4*s5*s3*s4*s2*s3*s1*s2, s1*s2*s3*s4*s5*s4*s3*s2, s1*s2*s3*s4*s5*s4*s3*s2*s1, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5, s5*s4*s5*s3*s4*s5*s2*s3*s4, s5*s4*s5*s3*s4*s5*s2*s3*s4*s1*s2*s3, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s5*s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4*s5*s3*s4, s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s3*s4*s5*s2*s3*s4*s5, s4*s5*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s4*s3*s2, s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3, s3*s4*s5*s2*s3*s4*s5*s3*s4, s3*s4*s5*s2*s3*s4*s5, s3*s4*s5*s2*s3*s4, s3*s4*s5*s4*s3*s2, s3*s4*s5*s4*s3*s2*s1, s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s5*s2*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s4*s5*s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5, s2*s3*s4*s5*s4*s3*s2*s1, s3*s4*s5*s2*s3*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5*s2*s3*s4, s3*s4*s5*s2*s3*s4*s5*s1*s2*s3*s4*s5]

dissectors = [x for x in join_irreducibles]

In [7]:
######################## D5 ########################

#sum_J KL = 60

#bigrassmannians: 69

bigrassmannians = [s3, s3*s2, s3*s4, s3*s5, s3*s2*s1, s3*s4*s2*s3, s3*s5*s2*s3, s3*s5*s4*s3, s3*s4*s2*s3*s1*s2, s3*s5*s2*s3*s1*s2, s2, s2*s3, s2*s3*s4, s2*s3*s5, s2*s3*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s5*s1*s2*s3, s4, s4*s3, s4*s3*s2, s4*s3*s5, s4*s3*s2*s1, s4*s3*s5*s2*s3, s4*s3*s5*s2*s3*s1*s2, s5, s5*s3, s5*s3*s2, s5*s3*s4, s5*s3*s2*s1, s5*s3*s4*s2*s3, s5*s3*s4*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s5, s1*s2*s3*s5*s4*s3, s1*s2*s3*s5*s4*s3*s2, s1*s2*s3*s5*s4*s3*s2*s1, s5*s3*s4*s2*s3*s5, s5*s3*s4*s2*s3*s5*s1*s2*s3, s4*s3*s5*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3, s2*s3*s5*s4*s3*s2, s3*s4*s2*s3*s5*s3*s4*s2*s3, s3*s4*s2*s3*s5, s3*s5*s4*s2*s3, s3*s5*s2*s3*s4, s3*s5*s4*s3*s2, s3*s5*s4*s3*s2*s1, s3*s5*s4*s2*s3*s1*s2, s3*s4*s2*s3*s5*s1*s2*s3, s3*s5*s2*s3*s4*s1*s2*s3, s5*s3*s4*s2*s3*s5*s1*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s4*s3*s2*s1, s3*s4*s2*s3*s5*s3*s4*s2*s3*s1*s2, s3*s4*s2*s3*s5*s3*s4*s1*s2*s3, s3*s5*s2*s3*s4*s1*s2*s3*s5*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s5*s1*s2*s3*s4, s3*s5*s2*s3*s4*s1*s2*s3*s5, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3, s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s4*s1*s2*s3, s2*s3*s5*s1*s2*s3*s4, s2*s3*s5*s4*s3*s1*s2]

#join_irreducibles: 64

join_irreducibles = [s3, s2, s4, s5, s1, s2*s3, s4*s3, s5*s3, s1*s2, s2*s3*s4, s4*s3*s2, s2*s3*s5, s5*s3*s2, s4*s3*s5, s5*s3*s4, s4*s3*s2*s1, s1*s2*s3, s5*s3*s2*s1, s3*s2, s3*s2*s1, s1*s2*s3*s4, s1*s2*s3*s5, s3*s4, s3*s4*s2*s3, s3*s4*s2*s3*s1*s2, s3*s5, s3*s5*s2*s3, s3*s5*s2*s3*s1*s2, s3*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s5*s1*s2*s3, s3*s4*s2*s3*s5*s3*s4*s2*s3, s3*s5*s4*s3*s2, s3*s5*s2*s3*s4, s3*s4*s2*s3*s5, s2*s3*s5*s4*s3*s2, s2*s3*s5*s4*s3, s3*s5*s4*s3*s2*s1, s4*s3*s5*s2*s3*s4, s4*s3*s5*s2*s3, s4*s3*s5*s2*s3*s1*s2, s5*s3*s4*s2*s3*s5, s5*s3*s4*s2*s3, s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s1*s2*s3*s4, s3*s4*s2*s3*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3, s3*s5*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s5*s1*s2*s3, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3*s1*s2, s2*s3*s5*s4*s3*s2*s1, s1*s2*s3*s5*s4*s3, s1*s2*s3*s5*s4*s3*s2, s1*s2*s3*s5*s4*s3*s2*s1, s5*s3*s4*s2*s3*s5*s1*s2*s3, s4*s3*s5*s2*s3*s4*s1*s2*s3, s5*s3*s4*s2*s3*s5*s1*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3*s5, s3*s5*s2*s3*s4*s1*s2*s3*s5*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s5*s1*s2*s3*s4, s3*s5*s2*s3*s4*s1*s2*s3*s5]


#dissectors: 52

dissectors = [s3, s3*s2, s3*s4, s3*s5, s3*s2*s1, s2, s2*s3, s2*s3*s4, s2*s3*s5, s2*s1, s2*s3*s1*s2, s4, s4*s3, s4*s3*s2, s4*s3*s5, s4*s3*s2*s1, s4*s3*s5*s2*s3, s4*s3*s5*s2*s3*s1*s2, s5, s5*s3, s5*s3*s2, s5*s3*s4, s5*s3*s2*s1, s5*s3*s4*s2*s3, s5*s3*s4*s2*s3*s1*s2, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s5, s1*s2*s3*s5*s4*s3, s1*s2*s3*s5*s4*s3*s2, s1*s2*s3*s5*s4*s3*s2*s1, s5*s3*s4*s2*s3*s5, s5*s3*s4*s2*s3*s5*s1*s2*s3, s4*s3*s5*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3, s2*s3*s5*s4*s3*s2, s3*s4*s2*s3*s5, s3*s5*s2*s3*s4, s3*s5*s4*s3*s2*s1, s5*s3*s4*s2*s3*s5*s1*s2*s3*s4, s4*s3*s5*s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s4*s3*s2*s1, s3*s4*s2*s3*s5*s3*s4*s2*s3*s1*s2, s3*s5*s2*s3*s4*s1*s2*s3*s5*s2*s3*s4*s1*s2*s3, s3*s4*s2*s3*s5*s1*s2*s3*s4, s3*s5*s2*s3*s4*s1*s2*s3*s5, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s1*s2*s3*s5*s3*s4*s2*s3, s2*s3*s4*s1*s2*s3*s5, s2*s3*s5*s1*s2*s3*s4]


In [11]:
######################## D6 ########################

#sum_J KL = 110

#bigrassmannians: 135

bigrassmannians = [s3, s3*s2, s3*s4, s3*s2*s1, s3*s4*s5, s3*s4*s6, s3*s4*s2*s3, s3*s4*s6*s5*s4, s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4, s3*s4*s6*s2*s3*s4, s3*s4*s6*s5*s4*s3, s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s6*s2*s3*s4*s1*s2*s3, s2, s2*s3, s2*s3*s4, s2*s3*s4*s5, s2*s3*s4*s6, s2*s3*s4*s6*s5*s4, s2*s3*s4*s6*s5*s4*s3, s2*s1, s2*s3*s1*s2, s2*s3*s4*s1*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s6*s1*s2*s3*s4, s4, s4*s3, s4*s3*s2, s4*s3*s2*s1, s4*s5, s4*s5*s3*s4, s4*s5*s3*s4*s6, s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s3*s4*s6*s2*s3*s4, s4*s5*s3*s4*s6*s2*s3*s4*s1*s2*s3, s4*s6, s4*s6*s3*s4, s4*s6*s3*s4*s5, s4*s6*s3*s4*s2*s3, s4*s6*s3*s4*s2*s3*s1*s2, s4*s6*s3*s4*s5*s2*s3*s4, s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s6*s5*s3*s4, s4*s6*s5*s3*s4*s2*s3, s4*s6*s5*s3*s4*s2*s3*s1*s2, s4*s6*s5*s4, s4*s6*s5*s4*s3, s4*s6*s5*s4*s3*s2, s4*s6*s5*s4*s3*s2*s1, s4*s5*s3*s4*s6*s4*s5*s3*s4, s1, s1*s2, s1*s2*s3, s1*s2*s3*s4, s1*s2*s3*s4*s5, s1*s2*s3*s4*s6, s1*s2*s3*s4*s6*s5*s4, s1*s2*s3*s4*s6*s5*s4*s3, s5, s5*s4, s5*s4*s3, s5*s4*s3*s2, s5*s4*s3*s2*s1, s5*s4*s6, s5*s4*s6*s3*s4, s5*s4*s6*s3*s4*s5, s5*s4*s6*s3*s4*s2*s3, s5*s4*s6*s3*s4*s2*s3*s1*s2, s5*s4*s6*s3*s4*s5*s2*s3*s4, s5*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s6, s6*s4, s6*s4*s3, s6*s4*s3*s2, s6*s4*s3*s2*s1, s6*s4*s5, s6*s4*s5*s3*s4, s6*s4*s5*s3*s4*s6, s6*s4*s5*s3*s4*s2*s3, s6*s4*s5*s3*s4*s2*s3*s1*s2, s6*s4*s5*s3*s4*s6*s2*s3*s4, s6*s4*s5*s3*s4*s6*s2*s3*s4*s1*s2*s3, s1*s2*s3*s4*s6*s5*s4*s3*s2*s1, s1*s2*s3*s4*s6*s5*s4*s3*s2, s5*s4*s6*s3*s4*s5*s2*s3*s4*s6, s5*s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4, s6*s4*s5*s3*s4*s6*s2*s3*s4*s5, s6*s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4, s5*s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5, s6*s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6, s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2, s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3, s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4, s2*s3*s4*s5*s1*s2*s3*s4*s6, s2*s3*s4*s6*s5*s1*s2*s3*s4, s2*s3*s4*s6*s1*s2*s3*s4*s5, s2*s3*s4*s6*s5*s4*s1*s2*s3, s2*s3*s4*s6*s5*s4*s3*s1*s2, s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4, s4*s5*s3*s4*s6*s4*s5*s3*s4*s2*s3, s4*s5*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2, s4*s5*s3*s4*s6*s4*s5*s2*s3*s4, s4*s5*s3*s4*s6*s4*s5*s2*s3*s4*s1*s2*s3, s4*s6*s3*s4*s5*s2*s3*s4*s6, s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4, s4*s5*s3*s4*s6*s2*s3*s4*s5, s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4, s2*s3*s4*s6*s5*s4*s3*s2, s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3, s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4, s3*s4*s5*s2*s3*s4*s6, s3*s4*s6*s5*s2*s3*s4, s3*s4*s6*s2*s3*s4*s5, s3*s4*s6*s5*s4*s2*s3, s3*s4*s6*s5*s4*s3*s2, s3*s4*s6*s5*s4*s3*s2*s1, s3*s4*s6*s5*s4*s2*s3*s1*s2, s3*s4*s6*s5*s2*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4, s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4, s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s1*s2*s3*s4, s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5, s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6, s2*s3*s4*s6*s5*s4*s3*s2*s1, s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2, s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s1*s2*s3, s3*s4*s5*s2*s3*s4*s6*s4*s5*s1*s2*s3*s4, s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3, s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4, s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5, s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6]

#join_irreducibles: 120

join_irreducibles = [s6,  s5,  s4,  s3,  s2,  s1,  s6*s4,  s5*s4,  s4*s6,  s4*s5,  s4*s3,  s3*s4,  s3*s2,  s2*s3,  s2*s1,  s1*s2,  s6*s4*s5,  s6*s4*s3,  s5*s4*s6,  s5*s4*s3,  s4*s3*s2,  s3*s4*s6,  s3*s4*s5,  s3*s2*s1,  s2*s3*s4,  s1*s2*s3,  s6*s4*s3*s2,  s5*s4*s3*s2,  s4*s6*s5*s4,  s4*s6*s3*s4,  s4*s5*s3*s4,  s4*s3*s2*s1,  s3*s4*s2*s3,  s2*s3*s4*s6,  s2*s3*s4*s5,  s2*s3*s1*s2,  s1*s2*s3*s4,  s6*s4*s5*s3*s4,  s6*s4*s3*s2*s1,  s5*s4*s6*s3*s4,  s5*s4*s3*s2*s1,  s4*s5*s3*s4*s6,  s4*s6*s3*s4*s5,  s4*s6*s5*s4*s3,  s3*s4*s6*s5*s4,  s1*s2*s3*s4*s6,  s1*s2*s3*s4*s5,  s6*s4*s5*s3*s4*s6,  s5*s4*s6*s3*s4*s5,  s4*s6*s3*s4*s2*s3,  s4*s5*s3*s4*s2*s3,  s4*s6*s5*s4*s3*s2,  s3*s4*s6*s2*s3*s4,  s3*s4*s5*s2*s3*s4,  s3*s4*s6*s5*s4*s3,  s3*s4*s2*s3*s1*s2,  s2*s3*s4*s6*s5*s4,  s2*s3*s4*s1*s2*s3,  s6*s4*s5*s3*s4*s2*s3,  s5*s4*s6*s3*s4*s2*s3,  s4*s6*s5*s4*s3*s2*s1,  s3*s4*s5*s2*s3*s4*s6,  s3*s4*s6*s2*s3*s4*s5,  s3*s4*s6*s5*s4*s3*s2,  s2*s3*s4*s6*s5*s4*s3,  s1*s2*s3*s4*s6*s5*s4,  s4*s6*s3*s4*s5*s2*s3*s4,  s4*s5*s3*s4*s6*s2*s3*s4,  s4*s6*s3*s4*s2*s3*s1*s2,  s4*s5*s3*s4*s2*s3*s1*s2,  s3*s4*s6*s5*s4*s3*s2*s1,  s2*s3*s4*s6*s1*s2*s3*s4,  s2*s3*s4*s5*s1*s2*s3*s4,  s2*s3*s4*s6*s5*s4*s3*s2,  s1*s2*s3*s4*s6*s5*s4*s3,  s6*s4*s5*s3*s4*s6*s2*s3*s4,  s6*s4*s5*s3*s4*s2*s3*s1*s2,  s5*s4*s6*s3*s4*s5*s2*s3*s4,  s5*s4*s6*s3*s4*s2*s3*s1*s2,  s4*s6*s3*s4*s5*s2*s3*s4*s6,  s4*s5*s3*s4*s6*s2*s3*s4*s5,  s4*s5*s3*s4*s6*s4*s5*s3*s4,  s3*s4*s6*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s2*s3*s4*s1*s2*s3,  s2*s3*s4*s5*s1*s2*s3*s4*s6,  s2*s3*s4*s6*s1*s2*s3*s4*s5,  s2*s3*s4*s6*s5*s4*s3*s2*s1,  s1*s2*s3*s4*s6*s5*s4*s3*s2,  s6*s4*s5*s3*s4*s6*s2*s3*s4*s5,  s5*s4*s6*s3*s4*s5*s2*s3*s4*s6,  s1*s2*s3*s4*s6*s5*s4*s3*s2*s1,  s4*s5*s3*s4*s6*s4*s5*s3*s4*s2*s3,  s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s4*s5*s3*s4*s6*s2*s3*s4*s1*s2*s3,  s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4,  s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4,  s6*s4*s5*s3*s4*s6*s2*s3*s4*s1*s2*s3,  s5*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6,  s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5,  s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4,  s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s4*s5*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3,  s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4,  s6*s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4,  s5*s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4,  s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6,  s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5,  s6*s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6,  s5*s4*s6*s3*s4*s5*s2*s3*s4*s6*s1*s2*s3*s4*s5,  s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4,  s3*s4*s5*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3,  s2*s3*s4*s5*s1*s2*s3*s4*s6*s4*s5*s3*s4*s2*s3*s1*s2,  s4*s6*s3*s4*s5*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4,  s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s3*s4*s5*s2*s3*s4*s1*s2*s3,  s4*s5*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4*s6*s2*s3*s4*s5*s1*s2*s3*s4]

#dissectors: ?

In [3]:
######################## E6 ########################

#sum_J KL = 156

#bigrassmannians: 232

bigrassmannians = [s6, s6*s5, s6*s5*s4, s6*s5*s4*s2, s6*s5*s4*s3, s6*s5*s4*s3*s1, s6*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5, s5*s6, s5*s4, s5*s6*s4*s5, s5*s4*s2, s5*s6*s4*s5*s2*s4, s5*s6*s4*s5*s2*s4*s3, s5*s6*s4*s5*s2*s4*s3*s1, s5*s4*s3, s5*s6*s4*s5*s3*s4, s5*s6*s4*s5*s3*s4*s2, s5*s6*s4*s5*s3*s2*s4, s5*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4, s5*s6*s4*s3*s2*s4*s5, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4, s4*s5, s4*s5*s6, s4*s2, s4*s5*s2*s4, s4*s5*s6*s2*s4*s5, s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3, s4*s5*s3*s4, s4*s5*s6*s3*s4*s5, s4*s5*s3*s4*s2, s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5, s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s1, s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3, s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4, s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2, s2*s4, s2*s4*s5, s2*s4*s5*s6, s2*s4*s3, s2*s4*s5*s3*s4, s2*s4*s5*s6*s3*s4*s5, s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s2*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3, s3*s4, s3*s4*s5, s3*s4*s5*s6, s3*s4*s2, s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s1, s3*s4*s5*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3, s3*s4*s5*s1*s3*s4, s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1, s1*s3, s1*s3*s4, s1*s3*s4*s5, s1*s3*s4*s5*s6, s1*s3*s4*s2, s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5, s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6]

#join_irreducibles: 182

join_irreducibles = [s1, s2, s3, s4, s5, s6, s1*s3, s2*s4, s3*s1, s3*s4, s4*s2, s4*s3, s4*s5, s5*s4, s5*s6, s6*s5, s1*s3*s4, s2*s4*s3, s2*s4*s5, s3*s4*s2, s3*s4*s5, s4*s3*s1, s4*s5*s6, s5*s4*s2, s5*s4*s3, s6*s5*s4, s1*s3*s4*s2, s1*s3*s4*s5, s2*s4*s3*s1, s2*s4*s5*s6, s3*s4*s1*s3, s3*s4*s5*s6, s4*s3*s2*s4, s4*s5*s2*s4, s4*s5*s3*s4, s5*s4*s3*s1, s5*s6*s4*s5, s6*s5*s4*s2, s6*s5*s4*s3, s1*s3*s4*s5*s6, s2*s4*s5*s3*s4, s3*s4*s5*s2*s4, s4*s5*s3*s4*s2, s4*s5*s2*s4*s3, s4*s3*s2*s4*s5, s5*s4*s3*s2*s4, s6*s5*s4*s3*s1, s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s2, s3*s4*s5*s2*s4*s3, s3*s4*s1*s3*s2*s4, s3*s4*s5*s1*s3*s4, s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5, s4*s5*s6*s3*s4*s5, s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4, s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4, s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s3*s4*s5, s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5, s5*s6*s4*s5*s3*s4*s2, s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4, s5*s6*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3, s6*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4]

#dissectors: ?

In [22]:
######################## E7 ########################

#sum_J KL = 399

#bigrassmannians: 945

bigrassmannians = [s1, s2, s3, s4, s5, s6, s7, s1*s3, s2*s4, s3*s1, s3*s4, s4*s2, s4*s3, s4*s5, s5*s4, s5*s6, s6*s5, s6*s7, s7*s6, s1*s3*s4, s2*s4*s3, s2*s4*s5, s3*s4*s2, s3*s4*s5, s4*s3*s1, s4*s5*s6, s5*s4*s2, s5*s4*s3, s5*s6*s7, s6*s5*s4, s7*s6*s5, s1*s3*s4*s2, s1*s3*s4*s5, s2*s4*s3*s1, s2*s4*s5*s6, s3*s4*s1*s3, s3*s4*s5*s6, s4*s3*s2*s4, s4*s5*s2*s4, s4*s5*s3*s4, s4*s5*s6*s7, s5*s4*s3*s1, s5*s6*s4*s5, s6*s5*s4*s2, s6*s5*s4*s3, s6*s7*s5*s6, s7*s6*s5*s4, s1*s3*s4*s5*s6, s2*s4*s5*s3*s4, s2*s4*s5*s6*s7, s3*s4*s5*s2*s4, s3*s4*s5*s6*s7, s4*s5*s3*s4*s2, s4*s5*s2*s4*s3, s4*s5*s3*s2*s4, s4*s3*s2*s4*s5, s5*s4*s3*s2*s4, s6*s5*s4*s3*s1, s7*s6*s5*s4*s2, s7*s6*s5*s4*s3, s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s7, s2*s4*s5*s3*s4*s2, s3*s4*s5*s2*s4*s3, s3*s4*s1*s3*s2*s4, s3*s4*s5*s1*s3*s4, s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5, s4*s5*s6*s3*s4*s5, s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4, s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6, s6*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5, s7*s6*s5*s4*s3*s1, s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s3*s4*s5, s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s1*s3*s4*s2, s3*s4*s5*s2*s4*s1*s3, s3*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5, s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s2, s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s2*s4, s5*s6*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4, s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s6*s7*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6, s5*s6*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5, s5*s6*s7*s4*s5*s6*s3*s4*s5, s5*s6*s7*s4*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s2, s6*s7*s5*s6*s4*s5*s2*s4*s3, s6*s7*s5*s6*s4*s5*s3*s2*s4, s6*s7*s5*s6*s4*s3*s2*s4*s5, s6*s7*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s3, s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4]

#join_irreducibles: 528

join_irreducibles = [s1, s2, s3, s4, s5, s6, s7, s1*s3, s2*s4, s3*s1, s3*s4, s4*s2, s4*s3, s4*s5, s5*s4, s5*s6, s6*s5, s6*s7, s7*s6, s1*s3*s4, s2*s4*s3, s2*s4*s5, s3*s4*s2, s3*s4*s5, s4*s3*s1, s4*s5*s6, s5*s4*s2, s5*s4*s3, s5*s6*s7, s6*s5*s4, s7*s6*s5, s1*s3*s4*s2, s1*s3*s4*s5, s2*s4*s3*s1, s2*s4*s5*s6, s3*s4*s1*s3, s3*s4*s5*s6, s4*s3*s2*s4, s4*s5*s2*s4, s4*s5*s3*s4, s4*s5*s6*s7, s5*s4*s3*s1, s5*s6*s4*s5, s6*s5*s4*s2, s6*s5*s4*s3, s6*s7*s5*s6, s7*s6*s5*s4, s1*s3*s4*s5*s6, s2*s4*s5*s3*s4, s2*s4*s5*s6*s7, s3*s4*s5*s2*s4, s3*s4*s5*s6*s7, s4*s5*s3*s4*s2, s4*s5*s2*s4*s3, s4*s3*s2*s4*s5, s5*s4*s3*s2*s4, s6*s5*s4*s3*s1, s7*s6*s5*s4*s2, s7*s6*s5*s4*s3, s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s7, s2*s4*s5*s3*s4*s2, s3*s4*s5*s2*s4*s3, s3*s4*s1*s3*s2*s4, s3*s4*s5*s1*s3*s4, s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5, s4*s5*s6*s3*s4*s5, s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4, s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6, s6*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5, s7*s6*s5*s4*s3*s1, s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s3*s4*s5, s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s2, s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4, s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s6*s7*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6, s5*s6*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5, s5*s6*s7*s4*s5*s6*s3*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s2, s6*s7*s5*s6*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4]

#dissectors: ?



In [ ]:
######################## E8 ########################

#sum_J KL = ?

#bigrassmannians: 8460

bigrassmannians = [s1, s2, s3, s4, s5, s6, s7, s8, s1*s3, s2*s4, s3*s1, s3*s4, s4*s2, s4*s3, s4*s5, s5*s4, s5*s6, s6*s5, s6*s7, s7*s6, s7*s8, s8*s7, s1*s3*s4, s2*s4*s3, s2*s4*s5, s3*s4*s2, s3*s4*s5, s4*s3*s1, s4*s5*s6, s5*s4*s2, s5*s4*s3, s5*s6*s7, s6*s5*s4, s6*s7*s8, s7*s6*s5, s8*s7*s6, s1*s3*s4*s2, s1*s3*s4*s5, s2*s4*s3*s1, s2*s4*s5*s6, s3*s4*s1*s3, s3*s4*s5*s6, s4*s3*s2*s4, s4*s5*s2*s4, s4*s5*s3*s4, s4*s5*s6*s7, s5*s4*s3*s1, s5*s6*s4*s5, s5*s6*s7*s8, s6*s5*s4*s2, s6*s5*s4*s3, s6*s7*s5*s6, s7*s6*s5*s4, s7*s8*s6*s7, s8*s7*s6*s5, s1*s3*s4*s5*s6, s2*s4*s5*s3*s4, s2*s4*s5*s6*s7, s3*s4*s5*s2*s4, s3*s4*s5*s6*s7, s4*s5*s3*s4*s2, s4*s5*s2*s4*s3, s4*s5*s3*s2*s4, s4*s3*s2*s4*s5, s4*s5*s6*s7*s8, s5*s4*s3*s2*s4, s6*s5*s4*s3*s1, s7*s6*s5*s4*s2, s7*s6*s5*s4*s3, s8*s7*s6*s5*s4, s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s7, s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s7*s8, s3*s4*s5*s2*s4*s3, s3*s4*s1*s3*s2*s4, s3*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8, s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5, s4*s5*s6*s3*s4*s5, s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4, s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6, s6*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5, s6*s7*s8*s5*s6*s7, s7*s6*s5*s4*s3*s1, s7*s8*s6*s7*s5*s6, s8*s7*s6*s5*s4*s2, s8*s7*s6*s5*s4*s3, s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s8, s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s3*s4*s5, s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s1*s3*s4*s2, s3*s4*s5*s2*s4*s1*s3, s3*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5, s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s2, s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s2*s4, s5*s6*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4, s8*s7*s6*s5*s4*s3*s1, s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s6*s7*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8, s5*s6*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5, s5*s6*s7*s4*s5*s6*s3*s4*s5, s5*s6*s7*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8, s6*s7*s5*s6*s4*s5*s3*s4*s2, s6*s7*s5*s6*s4*s5*s2*s4*s3, s6*s7*s5*s6*s4*s5*s3*s2*s4, s6*s7*s5*s6*s4*s3*s2*s4*s5, s6*s7*s5*s4*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8, s7*s8*s6*s7*s5*s6*s4*s5*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s1*s3, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7, s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2, s7*s8*s6*s7*s5*s6*s4*s5*s2*s4*s3, s7*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s3*s2*s4*s5, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7, s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s3, s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s6*s7*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3, s8*s7*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s3, s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s4, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s8*s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s7*s8*s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7, s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s8*s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s4*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s1*s3, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s2*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s4*s3*s2*s4*s1*s3, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s2*s4*s1*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s1*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s1*s3*s2*s4*s5*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s3*s4*s1*s3*s2*s4*s5*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4]

#join_irreducibles: 2060

join_irreducibles = [s1, s2, s3, s4, s5, s6, s7, s8, s1*s3, s2*s4, s3*s1, s3*s4, s4*s2, s4*s3, s4*s5, s5*s4, s5*s6, s6*s5, s6*s7, s7*s6, s7*s8, s8*s7, s1*s3*s4, s2*s4*s3, s2*s4*s5, s3*s4*s2, s3*s4*s5, s4*s3*s1, s4*s5*s6, s5*s4*s2, s5*s4*s3, s5*s6*s7, s6*s5*s4, s6*s7*s8, s7*s6*s5, s8*s7*s6, s1*s3*s4*s2, s1*s3*s4*s5, s2*s4*s3*s1, s2*s4*s5*s6, s3*s4*s1*s3, s3*s4*s5*s6, s4*s3*s2*s4, s4*s5*s2*s4, s4*s5*s3*s4, s4*s5*s6*s7, s5*s4*s3*s1, s5*s6*s4*s5, s5*s6*s7*s8, s6*s5*s4*s2, s6*s5*s4*s3, s6*s7*s5*s6, s7*s6*s5*s4, s7*s8*s6*s7, s8*s7*s6*s5, s1*s3*s4*s5*s6, s2*s4*s5*s3*s4, s2*s4*s5*s6*s7, s3*s4*s5*s2*s4, s3*s4*s5*s6*s7, s4*s5*s3*s4*s2, s4*s5*s2*s4*s3, s4*s3*s2*s4*s5, s4*s5*s6*s7*s8, s5*s4*s3*s2*s4, s6*s5*s4*s3*s1, s7*s6*s5*s4*s2, s7*s6*s5*s4*s3, s8*s7*s6*s5*s4, s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s7, s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s7*s8, s3*s4*s5*s2*s4*s3, s3*s4*s1*s3*s2*s4, s3*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8, s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5, s4*s5*s6*s3*s4*s5, s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4, s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6, s6*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5, s6*s7*s8*s5*s6*s7, s7*s6*s5*s4*s3*s1, s7*s8*s6*s7*s5*s6, s8*s7*s6*s5*s4*s2, s8*s7*s6*s5*s4*s3, s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s8, s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s3*s4*s5, s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s2, s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4, s8*s7*s6*s5*s4*s3*s1, s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s6*s7*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8, s5*s6*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5, s5*s6*s7*s4*s5*s6*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8, s6*s7*s5*s6*s4*s5*s3*s4*s2, s6*s7*s5*s6*s4*s5*s2*s4*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8, s7*s8*s6*s7*s5*s6*s4*s5*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s1*s3, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s2, s7*s8*s6*s7*s5*s6*s4*s5*s2*s4*s3, s7*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3, s8*s7*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s3, s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s4, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s8*s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s8*s6*s7*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s7*s8*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s7*s8*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s8*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s2*s4*s3*s1, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s5*s1*s3*s4*s2, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s3*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s4*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4, s1*s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s3*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s1*s3*s4*s2, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s6*s5*s4*s3*s2*s4*s5*s6*s1*s3*s4*s5*s2*s4*s3*s1, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s4*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5, s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s5*s4*s3*s2*s4*s5*s1*s3*s4*s2, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4, s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6, s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s3*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s3*s4*s1*s3*s2*s4*s5*s4*s3*s2*s4*s1*s3, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5, s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4, s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5, s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s7*s6*s5*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s8*s4*s3*s2*s4*s5*s6*s7*s1*s3*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s7*s4*s5*s6*s2*s4*s5*s3*s4*s1*s3*s2*s4*s5*s6*s4*s5*s3*s4*s1*s3*s2*s4*s5*s3*s4*s1*s3*s2*s4]

#dissectors: ?

In [24]:
from sage.interfaces.gap import  get_gap_memory_pool_size, set_gap_memory_pool_size
set_gap_memory_pool_size(94364842180)

In [21]:
i = 0
ma = len(W)
problems = []

for w in W:
#    JMw = set(JM(w))
    JM2w = set(JM2(w))
    JM3w = set(JM3(w))
    if JM2w != JM3w:
        problems.append(w)
        print("problem:")
        print(convert_to_123(w))
        print([eval(convert_to_123(x)) for x in JM2w  ]     )
        print([eval(convert_to_123(x)) for x in JM3w  ]     ) 
        print()
    i += 1
    print("%d/%d"%(i,ma))

print(problems)

1/23040
2/23040
3/23040
4/23040
5/23040
6/23040
7/23040
8/23040
9/23040
10/23040
11/23040
12/23040
13/23040
14/23040
15/23040
16/23040
17/23040
18/23040
19/23040
20/23040
21/23040
22/23040
23/23040
24/23040
25/23040
26/23040
27/23040
28/23040
29/23040
30/23040
31/23040
32/23040
33/23040
34/23040
35/23040
36/23040
37/23040
38/23040
39/23040
40/23040
41/23040
42/23040
43/23040
44/23040
45/23040
46/23040
47/23040
48/23040
49/23040
50/23040
51/23040
52/23040
53/23040
54/23040
55/23040
56/23040
57/23040
58/23040
59/23040
60/23040
61/23040
62/23040
63/23040
64/23040
65/23040
66/23040
67/23040
68/23040
69/23040
70/23040
71/23040
72/23040
73/23040
74/23040
75/23040
76/23040
77/23040
78/23040
79/23040
80/23040
81/23040
82/23040
83/23040
84/23040
85/23040
86/23040
87/23040
88/23040
89/23040
90/23040
91/23040
92/23040
93/23040
94/23040
95/23040
96/23040
97/23040
98/23040
99/23040
100/23040
101/23040
102/23040
103/23040
104/23040
105/23040
106/23040
107/23040
108/23040
109/23040
110/23040
111/2304

832/23040
833/23040
834/23040
835/23040
836/23040
837/23040
838/23040
839/23040
840/23040
841/23040
842/23040
843/23040
844/23040
845/23040
846/23040
847/23040
848/23040
849/23040
850/23040
851/23040
852/23040
853/23040
854/23040
855/23040
856/23040
857/23040
858/23040
859/23040
860/23040
861/23040
862/23040
863/23040
864/23040
865/23040
866/23040
867/23040
868/23040
869/23040
870/23040
871/23040
872/23040
873/23040
874/23040
875/23040
876/23040
877/23040
878/23040
879/23040
880/23040
881/23040
882/23040
883/23040
884/23040
885/23040
886/23040
887/23040
888/23040
889/23040
890/23040
891/23040
892/23040
893/23040
894/23040
895/23040
896/23040
897/23040
898/23040
899/23040
900/23040
901/23040
902/23040
903/23040
904/23040
905/23040
906/23040
907/23040
908/23040
909/23040
910/23040
911/23040
912/23040
913/23040
914/23040
915/23040
916/23040
917/23040
918/23040
919/23040
920/23040
921/23040
922/23040
923/23040
924/23040
925/23040
926/23040
927/23040
928/23040
929/23040
930/23040
931/23040


1569/23040
1570/23040
1571/23040
1572/23040
1573/23040
1574/23040
1575/23040
1576/23040
1577/23040
1578/23040
1579/23040
1580/23040
1581/23040
1582/23040
1583/23040
1584/23040
1585/23040
1586/23040
1587/23040
1588/23040
1589/23040
1590/23040
1591/23040
1592/23040
1593/23040
1594/23040
1595/23040
1596/23040
1597/23040
1598/23040
1599/23040
1600/23040
1601/23040
1602/23040
1603/23040
1604/23040
1605/23040
1606/23040
1607/23040
1608/23040
1609/23040
1610/23040
1611/23040
1612/23040
1613/23040
1614/23040
1615/23040
1616/23040
1617/23040
1618/23040
1619/23040
1620/23040
1621/23040
1622/23040
1623/23040
1624/23040
1625/23040
1626/23040
1627/23040
1628/23040
1629/23040
1630/23040
1631/23040
1632/23040
1633/23040
1634/23040
1635/23040
1636/23040
1637/23040
1638/23040
1639/23040
1640/23040
1641/23040
1642/23040
1643/23040
1644/23040
1645/23040
1646/23040
1647/23040
1648/23040
1649/23040
1650/23040
1651/23040
1652/23040
1653/23040
1654/23040
1655/23040
1656/23040
1657/23040
1658/23040
1659/23040

2315/23040
2316/23040
2317/23040
2318/23040
2319/23040
2320/23040
2321/23040
2322/23040
2323/23040
2324/23040
2325/23040
2326/23040
2327/23040
2328/23040
2329/23040
2330/23040
2331/23040
2332/23040
2333/23040
2334/23040
2335/23040
2336/23040
2337/23040
2338/23040
2339/23040
2340/23040
2341/23040
2342/23040
2343/23040
2344/23040
2345/23040
2346/23040
2347/23040
2348/23040
2349/23040
2350/23040
2351/23040
2352/23040
2353/23040
2354/23040
2355/23040
2356/23040
2357/23040
2358/23040
2359/23040
2360/23040
2361/23040
2362/23040
2363/23040
2364/23040
2365/23040
2366/23040
2367/23040
2368/23040
2369/23040
2370/23040
2371/23040
2372/23040
2373/23040
2374/23040
2375/23040
2376/23040
2377/23040
2378/23040
2379/23040
2380/23040
2381/23040
2382/23040
2383/23040
2384/23040
2385/23040
2386/23040
2387/23040
2388/23040
2389/23040
2390/23040
2391/23040
2392/23040
2393/23040
2394/23040
2395/23040
2396/23040
2397/23040
2398/23040
2399/23040
2400/23040
2401/23040
2402/23040
2403/23040
2404/23040
2405/23040

2965/23040
2966/23040
2967/23040
2968/23040
2969/23040
2970/23040
2971/23040
2972/23040
2973/23040
2974/23040
2975/23040
2976/23040
2977/23040
2978/23040
2979/23040
2980/23040
2981/23040
2982/23040
2983/23040
2984/23040
2985/23040
2986/23040
2987/23040
2988/23040
2989/23040
2990/23040
2991/23040
2992/23040
2993/23040
2994/23040
2995/23040
2996/23040
2997/23040
2998/23040
2999/23040
3000/23040
3001/23040
3002/23040
3003/23040
3004/23040
3005/23040
3006/23040
3007/23040
3008/23040
3009/23040
3010/23040
3011/23040
3012/23040
3013/23040
3014/23040
3015/23040
3016/23040
3017/23040
3018/23040
3019/23040
3020/23040
3021/23040
3022/23040
3023/23040
3024/23040
3025/23040
3026/23040
3027/23040
3028/23040
3029/23040
3030/23040
3031/23040
3032/23040
3033/23040
3034/23040
3035/23040
3036/23040
3037/23040
3038/23040
3039/23040
3040/23040
3041/23040
3042/23040
3043/23040
3044/23040
3045/23040
3046/23040
3047/23040
3048/23040
3049/23040
3050/23040
3051/23040
3052/23040
3053/23040
3054/23040
3055/23040

...

20186/23040
20187/23040
20188/23040
20189/23040
20190/23040
20191/23040
20192/23040
20193/23040
20194/23040
20195/23040
20196/23040
20197/23040
20198/23040
20199/23040
20200/23040
20201/23040
20202/23040
20203/23040
20204/23040
20205/23040
20206/23040
20207/23040
20208/23040
20209/23040
20210/23040
20211/23040
20212/23040
20213/23040
20214/23040
20215/23040
20216/23040
20217/23040
20218/23040
20219/23040
20220/23040
20221/23040
20222/23040
20223/23040
20224/23040
20225/23040
20226/23040
20227/23040
20228/23040
20229/23040
20230/23040
20231/23040
20232/23040
20233/23040
20234/23040
20235/23040
20236/23040
20237/23040
20238/23040
20239/23040
20240/23040
20241/23040
20242/23040
20243/23040
20244/23040
20245/23040
20246/23040
20247/23040
20248/23040
20249/23040
20250/23040
20251/23040
20252/23040
20253/23040
20254/23040
20255/23040
20256/23040
20257/23040
20258/23040
20259/23040
20260/23040
20261/23040
20262/23040
20263/23040
20264/23040
20265/23040
20266/23040
20267/23040
20268/23040
2026

20853/23040
problem:
34623451234634232
[346234512346, 3452346453423, 23465432]
[346234512346, 345234123, 3452346453423, 23465432]

20854/23040
20855/23040
20856/23040
20857/23040
20858/23040
20859/23040
20860/23040
20861/23040
20862/23040
20863/23040
20864/23040
20865/23040
20866/23040
20867/23040
20868/23040
20869/23040
20870/23040
problem:
34523461234534231
[345234612345, 34654321, 3452346453423]
[346234123, 345234612345, 34654321, 3452346453423]

20871/23040
problem:
34623451234634231
[346234512346, 34654321, 3452346453423]
[346234512346, 34654321, 345234123, 3452346453423]

20872/23040
20873/23040
problem:
3462345123463423
[346234512346, 3452346453423]
[346234512346, 345234123, 3452346453423]

20874/23040
20875/23040
20876/23040
20877/23040
20878/23040
20879/23040
20880/23040
20881/23040
20882/23040
20883/23040
20884/23040
20885/23040
20886/23040
20887/23040
20888/23040
20889/23040
20890/23040
20891/23040
20892/23040
20893/23040
20894/23040
20895/23040
20896/23040
20897/23040
20898

21498/23040
21499/23040
21500/23040
21501/23040
21502/23040
21503/23040
21504/23040
21505/23040
21506/23040
21507/23040
21508/23040
21509/23040
21510/23040
21511/23040
21512/23040
21513/23040
21514/23040
21515/23040
21516/23040
21517/23040
21518/23040
21519/23040
21520/23040
21521/23040
21522/23040
21523/23040
21524/23040
21525/23040
21526/23040
21527/23040
21528/23040
21529/23040
21530/23040
21531/23040
21532/23040
21533/23040
21534/23040
21535/23040
21536/23040
21537/23040
21538/23040
21539/23040
21540/23040
21541/23040
21542/23040
21543/23040
21544/23040
21545/23040
21546/23040
21547/23040
21548/23040
21549/23040
21550/23040
21551/23040
21552/23040
21553/23040
21554/23040
21555/23040
21556/23040
21557/23040
21558/23040
21559/23040
21560/23040
21561/23040
21562/23040
21563/23040
21564/23040
21565/23040
21566/23040
21567/23040
21568/23040
21569/23040
21570/23040
21571/23040
21572/23040
21573/23040
21574/23040
21575/23040
21576/23040
21577/23040
21578/23040
21579/23040
21580/23040
2158

22158/23040
22159/23040
22160/23040
22161/23040
22162/23040
22163/23040
22164/23040
22165/23040
22166/23040
22167/23040
22168/23040
22169/23040
22170/23040
22171/23040
22172/23040
22173/23040
22174/23040
22175/23040
22176/23040
22177/23040
22178/23040
22179/23040
22180/23040
22181/23040
22182/23040
22183/23040
22184/23040
22185/23040
22186/23040
22187/23040
22188/23040
22189/23040
22190/23040
22191/23040
22192/23040
22193/23040
22194/23040
22195/23040
22196/23040
22197/23040
22198/23040
22199/23040
22200/23040
22201/23040
22202/23040
22203/23040
22204/23040
22205/23040
22206/23040
22207/23040
22208/23040
22209/23040
22210/23040
22211/23040
22212/23040
22213/23040
22214/23040
22215/23040
22216/23040
22217/23040
22218/23040
22219/23040
22220/23040
22221/23040
22222/23040
22223/23040
22224/23040
22225/23040
22226/23040
22227/23040
22228/23040
22229/23040
22230/23040
22231/23040
22232/23040
22233/23040
22234/23040
22235/23040
22236/23040
22237/23040
22238/23040
22239/23040
22240/23040
2224

22842/23040
22843/23040
22844/23040
22845/23040
22846/23040
22847/23040
22848/23040
22849/23040
22850/23040
22851/23040
22852/23040
22853/23040
22854/23040
22855/23040
22856/23040
22857/23040
22858/23040
22859/23040
22860/23040
22861/23040
22862/23040
22863/23040
22864/23040
22865/23040
22866/23040
22867/23040
22868/23040
22869/23040
22870/23040
22871/23040
22872/23040
22873/23040
22874/23040
22875/23040
22876/23040
22877/23040
22878/23040
22879/23040
22880/23040
22881/23040
22882/23040
22883/23040
22884/23040
22885/23040
22886/23040
22887/23040
22888/23040
22889/23040
22890/23040
22891/23040
22892/23040
22893/23040
22894/23040
22895/23040
22896/23040
22897/23040
22898/23040
22899/23040
22900/23040
22901/23040
22902/23040
22903/23040
22904/23040
22905/23040
22906/23040
22907/23040
22908/23040
22909/23040
22910/23040
22911/23040
22912/23040
22913/23040
22914/23040
22915/23040
22916/23040
22917/23040
22918/23040
22919/23040
22920/23040
22921/23040
22922/23040
22923/23040
22924/23040
2292

In [ ]:
for w in problems[81:]:
#    JMw = set(JM(w))
    JM2w = set(JM2(w))
#    JM3w = set(JM3(w))
#    print(convert_to_123(w))
#    print("%s - %s"%(DL(w),DR(w)))
    
#    print([eval(convert_to_123(x)) for x in JMw  ]     )
#    print(eval(convert_to_123(w)) == join(JMw))
    
#    print([eval(convert_to_123(x)) for x in JM2w  ]     )
    print(eval(convert_to_123(w)) == join(JM2w))
    
#    print([eval(convert_to_123(x)) for x in JM3w  ]     )
#    print(eval(convert_to_123(w)) == join(JM3w))
#    print()

True
True
True
True
True
True
True
True
True


In [26]:
def l(x):
    return x.length()

def add_length(s):
    return s+"(%d)"%len(s)

J = { p : [] for p in cartesian_product([range(1,n+1),range(1,n+1)]) }

for x in join_irreducibles:
    J[ ( eval(convert_to_123(list(DL(x))[0])), eval(convert_to_123(list(DR(x))[0]))) ].append(x)
    
B = { p : [] for p in cartesian_product([range(1,n+1),range(1,n+1)]) }

for x in bigrassmannians:
    B[ ( eval(convert_to_123(list(DL(x))[0])), eval(convert_to_123(list(DR(x))[0]))) ].append(x)

for key in J:
    J[key].sort(key=l)

for key in B:
    B[key].sort(key=l)
    

In [56]:
for pair in [(4,4)]:
    
    print("\nJoins of JI in B_%s%s:\n"%pair)
    
    for S in Subsets(J[pair],3):

        comparable = False

        for s in Subsets(S,2):
            x, y = list(s)
            if x.bruhat_le(y) or y.bruhat_le(x):
                comparable = True
                break

        if comparable == True:
            continue

        else:
            sort_S = list(S)
            sort_S.sort(key=l)
            a,b,c = sort_S
            print("a = "+add_length(convert_to_123(a)) + ", b = "+add_length(convert_to_123(b)) + ", c = "+add_length(convert_to_123(c)))
            ab = join_B([a,b])
            ac = join_B([a,c])
            bc = join_B([b,c])
            if ab == ac and ac == bc:
                abc = ab
            else:
                abc = join_B([a,b,c])
                        
            print("ab = %s, ac = %s, bc = %s, abc = %s" %(add_length(str(ab)), 
                                                   add_length(str(ac)), 
                                                   add_length(str(bc)), 
                                                   add_length(str(abc)) ))
            if ab == ac and ac == bc:
                print("D-triple!")
            print()


Joins of JI in B_44:

a = 4534(4), b = 4324(4), c = 4524(4)
ab = 45324(5), ac = 45324(5), bc = 45324(5), abc = 45324(5)
D-triple!

a = 4324(4), b = 4524(4), c = 456345134(9)
ab = 45324(5), ac = 0(1), bc = 0(1), abc = 0(1)

a = 45341324(8), b = 43245134(8), c = 45624534(8)
ab = 4324541324(10), ac = 0(1), bc = 4563245134(10), abc = 45632451324(11)

a = 45634524(8), b = 43245134(8), c = 45624534(8)
ab = 0(1), ac = 4563245324(10), bc = 4563245134(10), abc = 45632451324(11)

a = 43245134(8), b = 45624534(8), c = 456345134(9)
ab = 4563245134(10), ac = 4563245134(10), bc = 4563245134(10), abc = 4563245134(10)
D-triple!

a = 43245134(8), b = 45624534(8), c = 432454324(9)
ab = 4563245134(10), ac = 4324541324(10), bc = 4563245324(10), abc = 45632451324(11)

a = 43245134(8), b = 45624534(8), c = 45673456134524(14)
ab = 4563245134(10), ac = 0(1), bc = 0(1), abc = 0(1)

a = 45341324(8), b = 43245134(8), c = 45634524(8)
ab = 4324541324(10), ac = 4563451324(10), bc = 0(1), abc = 45632451324(11)

a =

ab = 4324541324(10), ac = 0(1), bc = 0(1), abc = 0(1)

a = 45341324(8), b = 432456134524(12), c = 4567245634524(13)
ab = 4324561345324(13), ac = 0(1), bc = 456732456134524(15), abc = 456732456134524(15)

a = 45341324(8), b = 432456134524(12), c = 4567345624534(13)
ab = 4324561345324(13), ac = 45673456245134(14), bc = 0(1), abc = 0(1)

a = 45341324(8), b = 432456134524(12), c = 45672456345134(14)
ab = 4324561345324(13), ac = 0(1), bc = 0(1), abc = 0(1)

a = 45341324(8), b = 432456134524(12), c = 432456432454324(15)
ab = 4324561345324(13), ac = 0(1), bc = 4324564132454324(16), abc = 4324564132454324(16)

a = 45341324(8), b = 432456134524(12), c = 4324567432456432454324(22)
ab = 4324561345324(13), ac = 0(1), bc = 0(1), abc = 0(1)

a = 45341324(8), b = 4567245634524(13), c = 4567345624534(13)
ab = 0(1), ac = 45673456245134(14), bc = 4567324563245324(16), abc = 0(1)

a = 45341324(8), b = 4567245634524(13), c = 45672456345134(14)
ab = 0(1), ac = 0(1), bc = 456724563451324(15), abc = 45672456

ab = 456732456134524(15), ac = 0(1), bc = 456724563451324(15), abc = 45673245613451324(17)

a = 432456134524(12), b = 4567245634524(13), c = 45673456134524(14)
ab = 456732456134524(15), ac = 456732456134524(15), bc = 456732456134524(15), abc = 456732456134524(15)
D-triple!

a = 432456134524(12), b = 4567245634524(13), c = 43245643245134(14)
ab = 456732456134524(15), ac = 4324564132451324(16), bc = 0(1), abc = 0(1)

a = 432456134524(12), b = 4567245634524(13), c = 45634132454324(14)
ab = 456732456134524(15), ac = 4324564132454324(16), bc = 0(1), abc = 45673245613245324(17)

a = 432456134524(12), b = 4567245634524(13), c = 453413245341324(15)
ab = 456732456134524(15), ac = 4324564132451324(16), bc = 0(1), abc = 0(1)

a = 432456134524(12), b = 4567245634524(13), c = 432456432454324(15)
ab = 456732456134524(15), ac = 4324564132454324(16), bc = 4567324563245324(16), abc = 45673245613245324(17)

a = 432456134524(12), b = 4567245634524(13), c = 45673456245341324(17)
ab = 456732456134524(15), 

ab = 4324564132454324(16), ac = 45673245634132454324(20), bc = 45673245634132454324(20), abc = 45673245634132454324(20)

a = 432456134524(12), b = 45673456245341324(17), c = 4534132456543245134(19)
ab = 0(1), ac = 45341324565432451324(20), bc = 4567341324564324541324(22), abc = 4567341324564324541324(22)

a = 432456134524(12), b = 45673456245341324(17), c = 4567245634132454324(19)
ab = 0(1), ac = 45673245634132454324(20), bc = 4567324563413245341324(22), abc = 4567324563413245341324(22)

a = 432456134524(12), b = 45673456245341324(17), c = 432456743245643245134(21)
ab = 0(1), ac = 0(1), bc = 43245674324564324541324(23), abc = 0(1)

a = 432456134524(12), b = 45673456245341324(17), c = 4324567432456432454324(22)
ab = 0(1), ac = 0(1), bc = 43245674324564324541324(23), abc = 0(1)

a = 432456134524(12), b = 4567245634132454324(19), c = 4534132456543245134(19)
ab = 45673245634132454324(20), ac = 45341324565432451324(20), bc = 0(1), abc = 0(1)

a = 432456134524(12), b = 4534132456543245134(19

ab = 4563413245341324(16), ac = 4563413245341324(16), bc = 4563413245341324(16), abc = 4563413245341324(16)
D-triple!

a = 456245341324(12), b = 45634132454324(14), c = 432456432454324(15)
ab = 4563413245341324(16), ac = 4324564324541324(16), bc = 4324564132454324(16), abc = 43245641324541324(17)

a = 456245341324(12), b = 45634132454324(14), c = 43245671345624534(17)
ab = 4563413245341324(16), ac = 0(1), bc = 4324567134563245324(19), abc = 43245671345632451324(20)

a = 456245341324(12), b = 45634132454324(14), c = 4324567432456134524(19)
ab = 4563413245341324(16), ac = 43245674324561345324(20), bc = 43245674324561345324(20), abc = 43245674324561345324(20)

a = 456245341324(12), b = 45634132454324(14), c = 4324567432456432454324(22)
ab = 4563413245341324(16), ac = 0(1), bc = 0(1), abc = 0(1)

a = 456245341324(12), b = 453413245341324(15), c = 432456432454324(15)
ab = 4563413245341324(16), ac = 4324564324541324(16), bc = 43245641324541324(17), abc = 43245641324541324(17)

a = 4562453413

...

ab = 45341324567534132456543245134(29), ac = 0(1), bc = 45341324567341324564324541324(29), abc = 4534132456753413245654324541324(31)

a = 456724534132456543245134(24), b = 453413245673413245643245134(27), c = 4324567432456453413245341324(28)
ab = 45341324567534132456543245134(29), ac = 432456745341324565341324541324(30), bc = 0(1), abc = 0(1)

a = 456724534132456543245134(24), b = 453413245673413245643245134(27), c = 4324567453413245634132454324(28)
ab = 45341324567534132456543245134(29), ac = 432456745341324565341324541324(30), bc = 0(1), abc = 0(1)

a = 456724534132456543245134(24), b = 453413245673413245643245134(27), c = 45624534132456453413245341324(29)
ab = 45341324567534132456543245134(29), ac = 456724534132456453413245341324(30), bc = 453413245675341324564132451324(30), abc = 4534132456753413245654132451324(31)

a = 456724534132456543245134(24), b = 453413245673413245643245134(27), c = 4534132456756453413245634132454324(34)
ab = 45341324567534132456543245134(29), ac = 453413245

ab = 45341324567534132456543245134(29), ac = 0(1), bc = 0(1), abc = 0(1)

a = 453413245673413245643245134(27), b = 453413245675432456543245134(27), c = 4324567453413245634132454324(28)
ab = 45341324567534132456543245134(29), ac = 0(1), bc = 0(1), abc = 0(1)

a = 453413245673413245643245134(27), b = 453413245675432456543245134(27), c = 45624534132456453413245341324(29)
ab = 45341324567534132456543245134(29), ac = 453413245675341324564132451324(30), bc = 4534132456753413245654132451324(31), abc = 4534132456753413245654132451324(31)

a = 453413245673413245643245134(27), b = 453413245675432456543245134(27), c = 45624534132456765432456134524(29)
ab = 45341324567534132456543245134(29), ac = 456245341324567654324564132451324(33), bc = 456245341324567654324564132451324(33), abc = 456245341324567654324564132451324(33)

a = 453413245673413245643245134(27), b = 453413245675432456543245134(27), c = 432456745624534132456453413245341324(36)
ab = 45341324567534132456543245134(29), ac = 0(1), bc = 0(1

ab = 0(1), ac = 456245341324567654324564132451324(33), bc = 456245341324567654324564132454324(33), abc = 4562453413245676543245641324541324(34)

a = 453413245675432456543245134(27), b = 4534132456734132456432454324(28), c = 432456745624534132456453413245341324(36)
ab = 0(1), ac = 0(1), bc = 0(1), abc = 0(1)
D-triple!

a = 453413245675432456543245134(27), b = 4324567432456453413245341324(28), c = 4324567453413245634132454324(28)
ab = 0(1), ac = 0(1), bc = 432456745341324565341324541324(30), abc = 0(1)

a = 453413245675432456543245134(27), b = 4324567432456453413245341324(28), c = 45624534132456453413245341324(29)
ab = 0(1), ac = 4534132456753413245654132451324(31), bc = 4324567453413245653413245341324(31), abc = 45341324567534132456541324541324(32)

a = 453413245675432456543245134(27), b = 4324567432456453413245341324(28), c = 45624534132456765432456134524(29)
ab = 0(1), ac = 456245341324567654324564132451324(33), bc = 456245341324567654324564132454324(33), abc = 45624534132456765432456

ab = 432456745341324565341324541324(30), ac = 0(1), bc = 453413245675341324564132454324(30), abc = 0(1)

a = 4324567432456453413245341324(28), b = 4534132456734132456432454324(28), c = 45624534132456453413245341324(29)
ab = 0(1), ac = 4324567453413245653413245341324(31), bc = 4534132456753413245641324541324(31), abc = 45341324567534132456541324541324(32)

a = 4324567432456453413245341324(28), b = 4534132456734132456432454324(28), c = 45624534132456765432456134524(29)
ab = 0(1), ac = 456245341324567654324564132454324(33), bc = 456245341324567654324564132454324(33), abc = 456245341324567654324564132454324(33)

a = 4324567453413245634132454324(28), b = 4534132456734132456432454324(28), c = 45624534132456453413245341324(29)
ab = 453413245675341324564132454324(30), ac = 432456745341324563413245341324(30), bc = 4534132456753413245641324541324(31), abc = 4534132456753413245641324541324(31)

a = 4324567453413245634132454324(28), b = 4534132456734132456432454324(28), c = 45624534132456765432456

a = 45624534132456765432456134524(29), b = 4534132456734132456453413245341324(34), c = 432456745624534132456453413245341324(36)
ab = 45624534132456765432456453413245341324(38), ac = 4562453413245676453413245654132451324(37), bc = 45341324567564534132456453413245341324(38), abc = 456245341324567645341324565341324541324(39)

a = 45624534132456765432456134524(29), b = 4534132456734132456453413245341324(34), c = 4567345624534132456724534132456543245134(40)
ab = 45624534132456765432456453413245341324(38), ac = 456734562453413245672453413245654132451324(42), bc = 456734562453413245672453413245654132451324(42), abc = 456734562453413245672453413245654132451324(42)

a = 45624534132456765432456134524(29), b = 4562453413245674534132456543245134(34), c = 4534132456756453413245634132454324(34)
ab = 4562453413245676453413245654132451324(37), ac = 4562453413245676453413245634132454324(37), bc = 0(1), abc = 456245341324567645341324565341324541324(39)

a = 45624534132456765432456134524(29), b = 4562453